In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
tf.random.set_seed(42)

pd.set_option('display.max_columns', 500) 

In [ ]:
DATA_IN_PATH = '/content/gdrive/MyDrive/캡스톤/version_6/version_6/'


# Load Data: DB1(2020.5.17).xlsx

In [ ]:
#data_jaun = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'digestive(jaundice)')
#data_hemate = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'hematemesis(digestive)')
#data_Folie_Nerve = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Folie_Nerve')
#data_musculoskeletal = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'musculoskeletal')
#data_kidney = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'kidney')
#data_respiratory = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'respiratory')
#data_circulator1 = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'circulator1')
#data_circulator2 = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Circulator2')
#data_para = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Para_Female_Child')
#data_digestive = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'digestive')
#others = pd.read_excel('결측치db.xls', sheet_name='Sheet2')

In [ ]:
data_jaun = pd.read_excel(DATA_IN_PATH+'DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'digestive(jaundice)')
data_hemate = pd.read_excel(DATA_IN_PATH+'DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'hematemesis(digestive)')
#data_Folie_Nerve = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Folie_Nerve')
data_musculoskeletal = pd.read_excel(DATA_IN_PATH+'DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'musculoskeletal')
data_kidney = pd.read_excel(DATA_IN_PATH+'DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'kidney')
data_respiratory = pd.read_excel(DATA_IN_PATH+'DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'respiratory')
data_circulator1 = pd.read_excel(DATA_IN_PATH+'DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'circulator1')
data_circulator2 = pd.read_excel(DATA_IN_PATH+'DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'Circulator2')
data_para = pd.read_excel(DATA_IN_PATH+'DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'Para_Female_Child')
data_digestive = pd.read_excel(DATA_IN_PATH+'DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'digestive')
#others = pd.read_excel('결측치db.xls', sheet_name='Sheet2')

# 1. Preprocessing

## 0) Select columns and fill diseases

In [ ]:
def organize_data(df):
  df.columns = df.iloc[0]
  df = df[1:]
  df = df.iloc[:, 6:]
  df['level5/diagnosis'] = df['level5/diagnosis'].fillna(method='ffill') 
  df.drop('No.case', axis=1, inplace=True)

  return df

In [ ]:
data_digestive = organize_data(data_digestive)
data_musculoskeletal = organize_data(data_musculoskeletal)
data_kidney = organize_data(data_kidney)
data_respiratory = organize_data(data_respiratory)
data_para = organize_data(data_para)
#data_Folie_Nerve = organize_data(data_Folie_Nerve)
data_circulator1 = organize_data(data_circulator1)
data_circulator1 = data_circulator1.iloc[:, :21]

data_circulator2 = data_circulator2.iloc[:, 1:]
data_circulator2['진단 질병'] = data_circulator2['진단 질병'].fillna(method='ffill')
data_circulator2.drop('No.case', axis=1, inplace=True) 

In [ ]:
data_hemate = organize_data(data_hemate)
data_jaun = organize_data(data_jaun)

data_hemate['Level1'] = 'hematemesis(digestive)'
data_jaun['Level1'] = 'digestive(jaundice)'

In [ ]:
data_digestive['Level1'] = 'digestive'
data_musculoskeletal['Level1'] = 'musculoskeletal'
data_kidney['Level1'] = 'kidney'
data_respiratory['Level1'] = 'respiratory'
data_para['Level1'] = 'para'
data_circulator1['Level1'] = 'circulator1'
data_circulator2['Level1'] = 'circulator2'

In [ ]:
print(data_hemate.shape)
print(data_jaun.shape)
print(data_digestive.shape)
print(data_musculoskeletal.shape)
print(data_kidney.shape)
print(data_respiratory.shape)
print(data_para.shape)
#print(data_Folie_Nerve.shape)
print(data_circulator1.shape)
print(data_circulator2.shape)

(360, 22)
(440, 22)
(421, 22)
(205, 22)
(246, 22)
(205, 22)
(355, 22)
(305, 22)
(218, 22)


In [ ]:
data_circulator2.columns = data_circulator1.columns

data = pd.concat([data_digestive, data_musculoskeletal, data_kidney,
                  data_respiratory, data_circulator1, #data_Folie_Nerve,
                  data_circulator2, data_para, data_jaun, data_hemate], axis=0)
data.sample()

,level5/diagnosis,Chief complaint,Sex,Age,Height,Weight,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력,Level1
41,Breast pain following the menstrual cycle,가슴이 아파요,여자,35,162,55,2년 전,양쪽,7~10일정도 지속,비슷함,주기적으로 아픔,"전체적으로 아픈느낌, NRS 6점",-,생리 시작할 때 가장 아프고 점점 나아짐,-,-,"술 (일주일 소주 한병), 담배 (-), 커피 (-)",-,"외상 (-), 수술 (-)",고혈압/당뇨병/결핵/간염 (-/-/-/-),"현재 생리중, 만삭 자연분만 1회, 만삭/조산/유산/생존 (1/-/-/1), 생리 주기적",para


In [ ]:
nan_data = pd.read_excel(DATA_IN_PATH + 'at_least_10_nan_data.xlsx', sheet_name = 'Sheet1')
nan_data.drop('Unnamed: 0', inplace=True, axis=1)
nan_data.head(1)

,level5/diagnosis,Chief complaint,Sex,Age,Height,Weight,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력,Level1
0,Reflux esophagitis,피 토했어요,여자,40,165,65.0,2달전,NaN,NaN,비슷함,처음,"붉은색, 덩어리(-), 음식물(-)","복통, 기침",NaN,특이사항 없음,NaN,NaN,NaN,NaN,NaN,정상 월경주기,digestive


In [ ]:
print(data.shape)
data = pd.concat([data, nan_data], axis=0)
print(data.shape)

(2755, 22)
(2953, 22)


## 1) Drop rows if more than half are nan

In [ ]:
data = data.fillna('-')

In [ ]:
def to_nan(x):
  if(x == '-'):
    x = np.nan
  if(x == '(-)'):
    x = np.nan
  if(x=='없음'):
    x = np.nan
  if(x=='특이사항 없음'):
    x = np.nan
  if(x=='이상 없음'):
    x = np.nan
  if(x=='.'):
    x = np.nan
    
  return x

In [ ]:
for i in range(len(data.columns)):
    data[data.columns[i]] = data.apply(lambda x : to_nan(x[data.columns[i]]) , axis = 1 )

In [ ]:
print('Before', data.shape)
data.dropna(thresh = data.shape[1]/2, axis = 0, inplace = True)
print('After', data.shape)

Before (2953, 22)
After (2238, 22)


In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
print(data.shape)
print(data['level5/diagnosis'].value_counts())

(2238, 22)
Rupture of esophageal varices     100
Gastric ulcer / duodenal ulcer     50
Chronic pancreatitis               50
Acute cholecystitis                50
Esophageal cancer                  50
                                 ... 
intraducal papilloma                1
마주신경성 실신                            1
fibrocystic change                  1
일차성 고콜레스테롤혈증                        1
백의 고혈압                              1
Name: level5/diagnosis, Length: 289, dtype: int64


In [ ]:
#idx_num = data[data['Height'].isnull()].index
#data = data.drop(idx_num)

## 2) Change Nan to blank text

In [ ]:
data = data.fillna('-')

In [ ]:
def to_nan(x):
  if(x == '-'):
    x = ''
  elif(x == '아니오'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '몰라요'):
    x = ''
  elif(x == '모릅니다'):
    x = ''
  elif(x == '모름'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '.'):
    x = ''    
  return x

In [ ]:
for i in range(len(data.columns)):
    data[data.columns[i]] = data.apply(lambda x : to_nan(x[data.columns[i]]) , axis = 1 )

In [ ]:
print(data.shape)

(2238, 22)


## 3) Check Target Data, Disease

In [ ]:
data['level5/diagnosis'].value_counts()

Rupture of esophageal varices     100
Gastric ulcer / duodenal ulcer     50
Chronic pancreatitis               50
Acute cholecystitis                50
Esophageal cancer                  50
                                 ... 
intraducal papilloma                1
마주신경성 실신                            1
fibrocystic change                  1
일차성 고콜레스테롤혈증                        1
백의 고혈압                              1
Name: level5/diagnosis, Length: 289, dtype: int64

In [ ]:
len(data['level5/diagnosis'].unique())

289

### Organize differently expressed diseases to the same expression diseases

In [ ]:
disease_kor = pd.read_excel(DATA_IN_PATH + 'ver6_new_words.xlsx', sheet_name = '질병')
disease_kor = disease_kor[['level5/diagnosis', '번역']]
disease_kor.head()

,level5/diagnosis,번역
0,Pseudoparathyroidism,가성 부갑상선 기능 저하증
1,Familial hypercholesterolemia,가족성 고콜레스테롤혈증
2,Familial short stature,가족형 저신장증
3,hepatitis,간염
4,Interstitial lung disease (ILD),간질성 폐질환


In [ ]:
data = pd.merge(data, disease_kor, how='left',on='level5/diagnosis')
data = data.drop_duplicates()
data.reset_index(drop=True, inplace=True)

data['번역'].fillna(data['level5/diagnosis'], inplace=True)
data['level5/diagnosis'] = data['번역']
data.drop('번역', axis=1, inplace=True)
data.sample()

,level5/diagnosis,Chief complaint,Sex,Age,Height,Weight,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력,Level1
2173,일류성 요실금,"""소변이 자꾸 새요""",여성,74,158,54.0,4년전,,,심해짐,처음,,,,,,,,,당뇨신경병증,폐경,kidney


In [ ]:
print(data.shape)
#print(disease_kor.shape)
#print(a.shape)

(2222, 22)


In [ ]:
#ind = data[data['level5/diagnosis'] == 'Gastritis, gastric ulcer, duodenal ulcer'].index
#data.loc[ind, data.columns[0]] = 'Gastric ulcer / duodenal ulcer'
#data[data['level5/diagnosis'] == 'Gastric ulcer / duodenal ulcer']

#ind = data[data['level5/diagnosis'] == 'Irritable bowel syndrome (IBS)'].index
#data.loc[ind, data.columns[0]] = 'Irritable bowel syndrome'

#ind = data[data['level5/diagnosis'] == 'Irritable Bowel Syndrome (IBS)'].index
#data.loc[ind, data.columns[0]] = 'Irritable bowel syndrome'

- Change some disease names in Circulator2

In [ ]:
idx = data[data['level5/diagnosis'] == '공황장애 / 과다환기증후군'].index#['level5/diagnosis'] = '공황장애'
data.loc[idx, 'level5/diagnosis'] = '공황장애'

idx = data[data['level5/diagnosis'] == '소화성궤양'].index
data.loc[idx, 'level5/diagnosis'] = '소화성 궤양'

idx = data[data['level5/diagnosis'] == '기립성 저혈압(혈량저하증 동반되어 있을 수)'].index
data.loc[idx, 'level5/diagnosis'] = '기립성 저혈압(혈량저하증 동반 가능)'

idx = data[data['level5/diagnosis'] == '대동맥박리'].index
data.loc[idx, 'level5/diagnosis'] = '대동맥 박리'

idx = data[data['level5/diagnosis'] == '역류성식도염'].index
data.loc[idx, 'level5/diagnosis'] = '역류성 식도염'

idx = data[data['level5/diagnosis'] == '거미막하출혈(SAH)'].index
data.loc[idx, 'level5/diagnosis'] = '지주막하 출혈'

idx = data[data['level5/diagnosis'] == '과다환기증'].index # 원문: Panic disorder / hyperventilation syndrome
data.loc[idx, 'level5/diagnosis'] = '공황장애'

idx = data[data['level5/diagnosis'] == '판막 질환(AR, MR 등)'].index
data.loc[idx, 'level5/diagnosis'] = '판막 질환'

idx = data[data['level5/diagnosis'] == '폐경후증후군'].index
data.loc[idx, 'level5/diagnosis'] = '폐경증후군'

idx = data[data['level5/diagnosis'] == '폐경증후군(폐경)'].index
data.loc[idx, 'level5/diagnosis'] = '폐경증후군'

idx = data[data['level5/diagnosis'] == '마주신경성 실신'].index
data.loc[idx, 'level5/diagnosis'] = '미주신경성 실신'

In [ ]:
len(data['level5/diagnosis'].unique())

227

### Drop disease which has less than 5 cases

In [ ]:
target_counts = data['level5/diagnosis'].value_counts().reset_index()
target_counts.columns = ['Disease', 'counts']
target_counts = target_counts[target_counts['counts'] < 5]
target_counts

,Disease,counts
225,신증후군,4
226,갈색세포종,3


In [ ]:
target_counts = data['level5/diagnosis'].value_counts().reset_index()
target_counts.columns = ['Disease', 'counts']
target_counts = target_counts[target_counts['counts'] >= 5]
target_counts

,Disease,counts
0,식도 정맥류 파열,100
1,소화성 궤양,95
2,역류성 식도염,64
3,식도암,50
4,만성 췌장염,50
...,...,...
220,저혈량증,5
221,악성 고혈암,5
222,이식거부증,5
223,방광경부폐색,5


In [ ]:
data = data[data['level5/diagnosis'].isin(target_counts['Disease'])]
data.reset_index(drop=True, inplace=True)

In [ ]:
print(data.shape)
print(data['level5/diagnosis'].value_counts())

(2215, 22)
식도 정맥류 파열    100
소화성 궤양        95
역류성 식도염       64
식도암           50
만성 췌장염        50
            ... 
저혈량증           5
악성 고혈암         5
이식거부증          5
방광경부폐색         5
일시적 요실금        5
Name: level5/diagnosis, Length: 225, dtype: int64


## 4) Calculate BMI and define Obesity with Weight and Height
- https://www.tongyeong.go.kr/health/01635/01898/01902.web

In [ ]:
# Correct Height error
idx = data[data['Height'] == '58세 여성이 소변을 자주 보고 소변을 참기 어려우며 소변 누러 화장실을 가면 미처 속옷을 내리기도 전에 소변이 흘러나오는 증상이 4년 이상 지속되어 내원하였다. 기침을 하면 소변이 약간 새며,밤에 자다가도 소변이 마려워 3번 정도 깬다고 한다. 소변이 속옷에 묻어 냄새 및 회음부 주위 피부염이 발생하며 상기 증상으로 사회생활도 힘들다고 한다. 특이 과거력은 없다.'].index
data.loc[idx, 'Height'] = ''
#print(data['Height'].iloc[850])
#data['Height'].iloc[850]=''
#print(data['Height'].iloc[850])

In [ ]:
data['Height'] = data['Height'].replace('', '0')
data['Weight'] = data['Weight'].replace('', '0')

data['Height'] = data['Height'].astype('int')
data['Weight'] = data['Weight'].astype('int')

In [ ]:
data['BMI'] = data['Weight'] / (data['Height']/100)**2
data['BMI'].fillna(-1, inplace=True)

In [ ]:
def define_obesity(x):
  if x < 0:
    return '알 수 없음'
  elif x < 20.0:
    return '저체중'
  elif x <= 24.0:
    return '정상'
  elif x <= 29.0:
    return '과체중'
  else:
    return '비만'

data['Obesity'] = data.apply(lambda x : define_obesity(x['BMI']), axis=1)

## 5) Change Age to Groups of Age

In [ ]:
data['Age'] = data['Age'].astype(str)

In [ ]:
import re
# Infant/Child's Grouping Reference: https://www.doctorsnews.co.kr/news/articleView.html?idxno=20917&replyAll=&reply_sc_order_by=I

def replace_age(x):

  if ('개월' in x) or ('달' in x) or ('개월' in x):
    #print(x)
    month_age = int(re.findall("\d+", x)[0])*4
    x = '생후 ' + str(month_age) + '주'
  
  if ('주' in x):
     week_age = int(re.findall("\d+", x)[0])
     if week_age <= 4:
       x = '신생아' # 태어나서 1개월까지(newborn)
     elif week_age <= 4*12:
       x = '영아' # 태어나서 1년까지(infant)
     else:
       x = '유아' # 만 1세부터 6세까지(child)
  else:
     x = x.replace('세', '')
     x = x[:2]
     x = int(x)#.astype(int)
     x = str(x//10*10)+'대'

  return x

data['Age'] = data.apply(lambda x : replace_age(x['Age']), axis=1)

In [ ]:
data['Age']

0       50대
1       30대
2       70대
3       10대
4       60대
       ... 
2210    50대
2211    30대
2212    60대
2213    60대
2214    10대
Name: Age, Length: 2215, dtype: object

## 6) Arrange Sex

In [ ]:
data['Sex'].value_counts()

남자      829
여자      691
남성      139
여성      130
F       129
M        92
남자       79
여자       63
남성       45
여자       15
          3
Name: Sex, dtype: int64

In [ ]:
def organize_sex(x):
  x = x.replace('F', '여자')
  x = x.replace('M', '남자')
  x = x.replace('여성', '여자')
  x = x.replace('남성', '남자')
  #x = x.lstrip()
  x = x.replace('여자 ', '여자')
  x = x.replace('여자  ', '여자')
  x = x.replace('남자 ', '남자')
  x = x.replace('남성 ', '남자')

  return x

data['Sex'] = organize_sex(data['Sex'])

In [ ]:
data['Sex'].value_counts()

남자    1184
여자    1028
         3
Name: Sex, dtype: int64

## 7) Remove Negative values(-)

In [ ]:
data.columns[10:21]

Index(['Experience', 'Character', 'Associated Sx.', 'Factor', 'Event',
       '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력'],
      dtype='object')

In [ ]:
#k = data.copy()

In [ ]:
# since regex doesn't support '+', change '+' to '#'
def to_new_positive_value(x):
  if('+' in x):
    x = x.replace('+', '#')#np.nan
    
  return x

for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
  data[m] = data.apply(lambda x : to_new_positive_value(x[m]) , axis = 1 )

In [ ]:
idx = data[data['Associated Sx.'] == '발열(#, 4주 전) 가래(#), 기침(#), 체중변화 (#, -4kg/2개월)'].index
data.loc[idx, 'Associated Sx.'] = '발열(#, 4주 전), 가래(#), 기침(#), 체중변화 (#, -4kg/2개월)'

idx = data[data['사회력'] == '술(#주) 흡연(#), 과식(#), 지방식(#)'].index
data.loc[idx, '사회력'] = '술(#주), 흡연(#), 과식(#), 지방식(#)'

In [ ]:
data['Associated Sx.'][0]

'[전신] 발열 / 식은땀 / 체중 감소 / 오한/ 피로감(-/-/-/-/-), [소화기] 구역 / 구토 / 식욕 부진 / 변비 / 설사(-/#, 2회 흰 물만 나옴 /-/-/-), 속쓰림, 혈변(-), [응급] 어지러움 / 갈증 / 소변량 감소 / 호흡곤란(#/#/#/-)'

In [ ]:
def remove_negative_values(x):
  
  #print('x',x,'1')
  # 1. To extract texts from brackets: (-/#, 2회 흰 물만 나옴 /-/-/-).
  brackets = re.findall('\((.*?)\)', x) # ['-/-/-/-/-', '-/#, 2회 흰 물만 나옴 /-/-/-', '-', '#/#/#/-']
  for i in range(len(brackets)):
    n = brackets[i].split('/') # ['-', '#, 2회 흰 물만 나옴 ', '-', '-', '-']

    for j in range(len(n)):
      if len(n[j])>1:
        tmp_str = n[j][2:] #  Behind '#, ' --> ' 2회 흰 물만 나옴 '
        if len(re.findall('\#', n[j]))==0: # '(#) (지금 복용 중)'
          n[j]=''
        else:
          n[j] = '#'   
        x = x.replace(brackets[i],'/'.join(n)) # Drawback: doesn't contain texts which was in brackets

  #if len(re.findall(r"(\d+)-?(\d+)", x)) > 0: # 하루 4-5회
  #  x = x.replace('-', '~')

  # 2. To remove negative values
  splited_x = x.split(',')
  for i in range(len(splited_x)):
    if ('및' in splited_x[i]):
      splited_x = x.split(',') and x.split('및')
      break

  #print(splited_x)
  for i in range(len(splited_x)):
    ne_or_po = re.findall('-|#',splited_x[i]) #.index('') ['#', '#', '#', '-']
    sym = splited_x[i].split('/') # [' [응급] 어지러움 ', ' 갈증 ', ' 소변량 감소 ', ' 호흡곤란(#', '#', '#', '-)']

    only_positive = ""
    #print(ne_or_po)
    for j in range(len(ne_or_po)):
      if ne_or_po[j] != '-':
        only_positive += sym[j]

      splited_x[i] = only_positive
        #print('@fin_splited_x', splited_x[i])
  x = (',').join(splited_x)
  #print('@@fin_x', x)

  return x

#def remove_negative_values(x):
#  tmp = x.split(',')
#  for i in range(len(tmp)):
#    if ('-' in tmp[i]):
#      tmp[i] = ''
#  x = ''.join(tmp)
#  return x
#for i in range(len(data.columns)):
#    data[data.columns[i]] = data.apply(lambda x : remove_negative_values(x[data.columns[i]]) , axis = 1 )

In [ ]:
for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
  data[m] = data.apply(lambda x : remove_negative_values(x[m]) , axis = 1 )

In [ ]:
data['Associated Sx.'][0]

', 구토 , 속쓰림,, [응급] 어지러움  갈증  소변량 감소 '

In [ ]:
def to_ori_new_positive_value(x):
  if('#' in x):
    x = x.replace('#', '+')#np.nan
    
  return x

for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
  data[m] = data.apply(lambda x : to_ori_new_positive_value(x[m]) , axis = 1 )

## 8) NRS scores to text

In [ ]:
#NRS 5점
#NRS: 5
#NRS 4~6 / NRS : 4~6

In [ ]:
def NRS_to_text(text):

  a = re.findall('NRS.*?점', text) #NRS 5점
  if len(a) == 0:
    a = re.findall('NRS.*?\d~\d', text) #NRS 4~6 / NRS : 4~6
  if len(a) == 0:
    a = re.findall('NRS.*?\d', text) #NRS: 5
  
  try:
    t = re.findall('[0-9]+', a[0])

    if (len(t) == 2):
      score = (int(t[0])+int(t[1]))/ 2
    else:
      score = int(t[0])

    if score >= 7:
      to_text = '심함'
    elif score >= 4:
      to_text = '중간'
    else:
      to_text = '약함'

    text = text.replace(a[0], to_text)
  
  except:
    pass

  return text

In [ ]:
#text = '뻐근한 통증, NRS : 1~6, 등이랑 목으로 뻗침'
text = '뻐근한 통증dd, NRS : 1~6, 등이랑 목으로 뻗침'

NRS_to_text(text)

'뻐근한 통증dd, 약함, 등이랑 목으로 뻗침'

In [ ]:
#data[data['Character'] == '쥐어짜는 느낌의 통증, NRS 7점, 턱과 목쪽으로 방사되는 통증, 왼팔로도 통증이 뻗치는 듯한 느낌 있음']
#data.iloc[872]['Character']

In [ ]:
for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
  data[m] = data.apply(lambda x : NRS_to_text(x[m]) , axis = 1 )

In [ ]:
#data.iloc[872]['Character']

## 9-1) Make a sentence

In [ ]:
data['All'] = (data['Chief complaint'].map(str) +'. '+ data['Age'].map(str) + '. '+
                data['Sex'].map(str) + '. '+
                data['Onset'].map(str) + '. '+ data['Location'].map(str) + '. ' +
                data['Duration'].map(str) + '. ' + data['Course'].map(str) + '. ' +
                data['Experience'].map(str) + '. ' + data['Character'].map(str) + '. ' +
                data['Associated Sx.'].map(str)+'. '+ data['Factor'].map(str) + '. ' +
                data['Event'].map(str) + '. ' + data['약물 투약력'].map(str) + '. ' +
                data['사회력'].map(str) + '. ' + data['가족력'].map(str) + '. ' +
                data['외상력'].map(str) + '. ' + data['과거력'].map(str) + '. ' +
                data['여성력'].map(str) + '. ' + data['Obesity']).apply(lambda row: row.strip())

In [ ]:
data['All']

0       배가 아파요. 50대. 남자. 1일전. 명치 부위. 지속. 심해짐. 이전에도 3차례...
1       갑자기 배가 너무 아파요. 30대. 남자. 오늘 새벽. 명치에서 시작해 현재 복부 ...
2       속이 쓰려요. 70대. 여자. 1달전. 명치 부위. 지속. 심해짐. . 속이 쓰림,...
3       속이 쓰려요. 10대. 남자. 2달전. 명치부위 . 지속. 심해짐. . 명치부위 압...
4       배가 아파요. 60대. 남자. 1시간전. 명치부위. 지속. 심해짐. . 명치 부위가...
                              ...                        
2210    콜레스테롤이 높대요. 50대. 남자. 1개월 전. . . . 처음. . 피곤, 식욕...
2211    콜레스테롤이 높대요. 30대. 여자. 1주 전. . . . 처음. . 체중변화(/1...
2212    콜레스테롤이 높대요. 60대. 여자. 1개월 전. . . . 처음. . . . . ...
2213    콜레스테롤이 높대요. 60대. 여자. 3개월 전. . . . 처음. . . . . ...
2214    생리를 안해요. 10대. 여자. 처음부터. . . . 처음. 초경 안옴. 턱이 짧음...
Name: All, Length: 2215, dtype: object

In [ ]:
data

,Height,Weight,Age,Chief complaint,Sex,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력,BMI,Obesity,All
0,180,75,30대,배가 아파요,남자,1일 전,명치 부위,지속,심해짐,"이전에도 3차례, 통증은 이번보다 약했음","칼로 찢기는 듯한 통증, NRS 8점, 방사통: 등으로 퍼짐","구토, 속쓰림, 어지러움, 갈증, 소변량 감소",오른쪽으로 돌아 누우면 완화,,,"술: 1주일 6~7번, 식사는 매우 불규칙",,,,,23.148148,정상,배 아프다 30 대다 1일 명치 부위 남자 심해지다 이전 에도 3 차례 통증 보다 ...


## 9) Check spelling

In [ ]:
def manual_spelling(x):
  # Need to update more!!
  x = x.replace('Live(', 'Liver(')
  x = x.replace('혈번', '혈변')
  x = x.replace('[회수] 회 / 일', '')
  x = x.replace('ㅊ음', '처음')
  x = x.replace('위엄', '위염')
  x = x.replace('소회기', '소화기')
  x = x.replace('소화쉐양', '소화궤양')
  x = x.replace('무릅', '무릎')
  x = x.replace('성광계', '성관계')
  x = x.replace('게속', '계속')
  x = x.replace('느김', '느낌')
  x = x.replace('불규칙하', '불규칙한')
  x = x.replace('뻗뻗', '뻣뻣')
  x = x.replace('부움', '부음')
  x = x.replace('피료', '피로')
  x = x.replace('들이마쉴', '들이마실')
  x = x.replace('(Olodaterol)', '')
  x = x.replace('체충', '체중')
  x = x.replace('속옥', '속옷')
  x = x.replace('아이업음', '아이없음')
  x = x.replace('재왕절개', '제왕절개')
  x = x.replace('얄', '열')
  x = x.replace('연 (', '열 (')
  x = x.replace('가끔식', '가끔씩')
  x = x.replace('붙성교', '성교')
  x = x.replace('월경량와', '월경량과')
  x = x.replace('호ㅡㅂ음', '호흡음')
  x = x.replace('오료계', '요로계')
  x = x.replace('분철성', '분출성')
  return x

In [ ]:
#for m in data.columns[6:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
#  data[m] = data.apply(lambda x : manual_spelling(x[m]) , axis = 1 )
data['All'] = manual_spelling(data['All'])

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-52c42wft
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-52c42wft


In [ ]:
data['All']

0       배가 아파요. 50대. 남자. 1일전. 명치 부위. 지속. 심해짐. 이전에도 3차례...
1       갑자기 배가 너무 아파요. 30대. 남자. 오늘 새벽. 명치에서 시작해 현재 복부 ...
2       속이 쓰려요. 70대. 여자. 1달전. 명치 부위. 지속. 심해짐. . 속이 쓰림,...
3       속이 쓰려요. 10대. 남자. 2달전. 명치부위 . 지속. 심해짐. . 명치부위 압...
4       배가 아파요. 60대. 남자. 1시간전. 명치부위. 지속. 심해짐. . 명치 부위가...
                              ...                        
2210    콜레스테롤이 높대요. 50대. 남자. 1개월 전. . . . 처음. . 피곤, 식욕...
2211    콜레스테롤이 높대요. 30대. 여자. 1주 전. . . . 처음. . 체중변화(/1...
2212    콜레스테롤이 높대요. 60대. 여자. 1개월 전. . . . 처음. . . . . ...
2213    콜레스테롤이 높대요. 60대. 여자. 3개월 전. . . . 처음. . . . . ...
2214    생리를 안해요. 10대. 여자. 처음부터. . . . 처음. 초경 안옴. 턱이 짧음...
Name: All, Length: 2215, dtype: object

In [ ]:
from tqdm.notebook import tqdm
from hanspell import spell_checker

def spelling_check(text):
  #print(text)
  result = spell_checker.check(text) 
  
  return result.as_dict()['checked'] 

In [ ]:
for i in tqdm(range(len(data))):
  data['All'][i] = spelling_check(data['All'][i])

  0%|          | 0/2215 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data['All']

0       배가 아파요. 50대. 남자. 1일 전. 명치 부위. 지속. 심해짐. 이전에도 3차...
1       갑자기 배가 너무 아파요. 30대. 남자. 오늘 새벽. 명치에서 시작해 현재 복부 ...
2       속이 쓰려요. 70대. 여자. 1달 전. 명치 부위. 지속. 심해짐. . 속이 쓰림...
3       속이 쓰려요. 10대. 남자. 2달 전. 명치 부위. 지속. 심해짐. . 명치 부위...
4       배가 아파요. 60대. 남자. 1시간 전. 명치 부위. 지속. 심해짐. . 명치 부...
                              ...                        
2210    콜레스테롤이 높대요. 50대. 남자. 1개월 전. . . . 처음. . 피곤, 식욕...
2211    콜레스테롤이 높대요. 30대. 여자. 1주 전. . . . 처음. . 체중 변화(/...
2212    콜레스테롤이 높대요. 60대. 여자. 1개월 전. . . . 처음. . . . . ...
2213    콜레스테롤이 높대요. 60대. 여자. 3개월 전. . . . 처음. . . . . ...
2214    생리를 안 해요. 10대. 여자. 처음부터. . . . 처음. 초경 안 옴. 턱이 ...
Name: All, Length: 2215, dtype: object

## 10) Change English words to Korean

In [ ]:
def replace_eng_to_kor(x):
  # Need to update more!!
  x = x.replace('Liver', '간염')
  x = x.replace('Hepatitis', '간염')
  x = x.replace('DM', '당뇨')
  x = x.replace('TBc', '결핵')
  x = x.replace('TB', '결핵')
  x = x.replace('HTN', '고혈압')
  x = x.replace('Dyslipidemia', '이상지질혈증')
  x = x.replace('dyslipidemia', '이상지질혈증')
  x = x.replace('소염진통제(NSAIDs)', '소염진통제')
  x = x.replace('NSAIDs', '소염진통제')
  x = x.replace('NSAID', '소염진통제')
  x = x.replace('LMP', '마지막 월경')
  x = x.replace("Murphy's sign", '숨을 들이쉬는 동안 오른쪽 갈비뼈 밑 인근 부위를 만지면 통증이 생기는 증상')
  x = x.replace('Valsartan', '고혈압약')
  x = x.replace('고혈압약(amlodipine)', '고혈압약')
  x = x.replace('항고혈압제(Amlodipine)', '고혈압약')
  x = x.replace('Amlodipine', '고혈압약')
  x = x.replace('amlodipine', '고혈압약')
  x = x.replace('Metformin', '비구아니드 계열의 경구용 혈당강하제, 제2형 당뇨병에 사용')
  x = x.replace('BID', '하루에 2번')
  x = x.replace('Bid', '하루에 2번')
  x = x.replace('ciprofloxacin', '항생제')
  x = x.replace('QD', '하루에 한번')
  x = x.replace('우울증약(SSRI)', '우울증약')
  x = x.replace('SSRI', '우울증약')
  x = x.replace('1T', '1알')
  x = x.replace('FAP', '가족성 용종증')
  x = x.replace('COPD', '만성 폐쇄성 폐질환')
  x = x.replace('Roux-Y op', '위우회술')
  x = x.replace('terminal ileum', '직장과 결장 및 소장의 마지막 일부')
  x = x.replace('Hyperlipidemia', '고지혈증')
  x = x.replace('혈압강하제(CCB, ACEi)', '혈압강하제')
  x = x.replace('C-sec', '제왕절개')
  x = x.replace('Non-smoking, Non-Alcohol', '술 담배 안함')
  x = x.replace('F / U / N / D / H / I / S', '빈뇨/절박뇨/야뇨/배뇨통/배뇨지연/요실금/소변줄기 변화')
  
  return x

In [ ]:
#for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
#  data[m] = data.apply(lambda x : replace_eng_to_kor(x[m]) , axis = 1 )

for i in tqdm(range(len(data))):
  data['All'][i] = replace_eng_to_kor(data['All'][i])

  0%|          | 0/2215 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


## 11) Change difficuly words to easy words

In [ ]:
def replace_diffi_to_easy(x):
  # Need to update more!!
  if x=='지속' or x=='지속 ' or x=='지속됨':
      x = '하루종일'
  x = x.replace('복압 배뇨', '소변을 볼 때 힘을 줘야함')#kidney
  x = x.replace('샅통증', '사타구니 통증')
  x = x.replace('다갈', '몹시 갈증이 남')
  x = x.replace('다음', '몹시 갈증이 나 물을 많이 마심')
  x = x.replace('다뇨', '소변량 증가')
  x = x.replace('뒷무직', '잔변감')
  x = x.replace('요의', '오줌이 마려운 느낌')
  x = x.replace('급성 요폐', '소변이 마려운데 힘줘도 안 나옴')
  x = x.replace('점적뇨', '소변 후 오줌이 몇 방울씩 속옷에 묻어남')
  x = x.replace('두부 외상', '머리를 다침')
  x = x.replace('두부 타격', '머리를 다침')
  x = x.replace('섬망', '환각, 초조함, 떨림')

  x = x.replace('복장뼈아래', '가슴 중앙 아래')#digestive
  x = x.replace('방사통', '퍼지는 통증')
  x = x.replace('반동압통', '눌렀다 떼면 아픔')
  x = x.replace('반발압통', '눌렀다 떼면 아픔')
  x = x.replace('압통점', '누르면 아픔')
  x = x.replace('압통', '누르면 아픔')
  x = x.replace('장음 감소', '장의 운동 소리가 감소')
  x = x.replace('장음감소', '장의 운동 소리가 감소')
  x = x.replace('장음 소실', '장의 운동 소리가 감소')
  x = x.replace('장음 증가', '장의 운동 소리가 증가')
  x = x.replace('장음증가', '장의 운동 소리가 증가')
  x = x.replace('박동성 종괴', '두근거리는 멍울, 혹')
  x = x.replace('종괴', '멍울, 혹')
  x = x.replace('덩이', '멍울, 혹')
  x = x.replace('두덩뼈', '골반뼈 중 일부')
  x = x.replace('공막', '흰자, 눈')
  x = x.replace('혈액응고지연', '피가 안멈춰요')
  x = x.replace('인두통', '목이 아픔')
  x = x.replace('항문 열상', '항문이 찢어져서 생긴 상처')
  x = x.replace('후중감(tenesmus)', '잔변감')
  x = x.replace('경부강직', '목근육이 굳어서 머리를 앞으로 구부릴 수 없음')
  x = x.replace('조음장애', '발음장애')
  x = x.replace('구음장애', '말하기 어려움')
  x = x.replace('추위불내성', '추위를 견디지 못함')
  x = x.replace('추위 불내성', '추위를 견디지 못함')
  x = x.replace('더위불내성', '더위를 견디지 못함')
  x = x.replace('더위 불내성', '더위를 견디지 못함')
  x = x.replace('야간발한', '밤에 땀이 남')
  x = x.replace('야간 발한', '밤에 땀이 남')
  x = x.replace('발한과다', '땀이 많이 남')
  x = x.replace('전신발한', '온몸에 땀이 남')
  x = x.replace('발한', '땀이 남')
  x = x.replace('와파린', '항응고제')
  x = x.replace('경추', '목')
  x = x.replace('요추', '허리')
  x = x.replace('폴립', '종양, 용종')
  x = x.replace('의식 소실', '의식 없어짐')
  x = x.replace('의식소실', '의식 없어짐')
  x = x.replace('황달 소실', '황달 없어짐')
  x = x.replace('유지', '비슷')
  x = x.replace('우상복부', '오른쪽 윗배')
  x = x.replace('우하복부', '오른쪽 아랫배')
  x = x.replace('우측복부', '오른쪽 배')
  x = x.replace('좌측복부', '왼쪽 배')
  x = x.replace('좌하복부', '왼쪽 아랫배')
  x = x.replace('좌상복부', '왼쪽 윗배')
  x = x.replace('하복부통', '아랫배 통증')
  x = x.replace('상복부', '윗배')
  x = x.replace('하복부', '아랫배')
  x = x.replace('심계항진', '심장이 계속 두근거려요')
  x = x.replace('분출성', '뿜어나오는 구토')
  x = x.replace('홍반', '붉은 반점')



  x = x.replace('작열감', '화끈거림')#musculoskeletal
  x = x.replace('광과민성', '빛에 민감')
  x = x.replace('간헐적', '주기적으로')#respiratory
  x = x.replace('간간히', '가끔씩')
  x = x.replace('혈성 콧물', '피가 섞인 콧물')
  x = x.replace('인후통', '목구멍이 아픔')
  x = x.replace('객혈', '피가 섞인 가래')
  x = x.replace('객담', '가래')

  #오목부종 오목 부종 안면부종 다리부종 하지부종 음낭부종 얼굴부종


  x = x.replace('편마비', '한 쪽 마비')#circulator2
  x = x.replace('메티마졸', '갑상선기능항진증 약')
  x = x.replace('희발월경', '월경, 생리 주기가 길다') 
  x = x.replace('측면', '옆')
  x = x.replace('무월경', '월경 안함')
  x = x.replace('지방식', '기름진 음식')
  x = x.replace('위약', '쇠약')
  x = x.replace('쇠약감', '힘이 없음')
  x = x.replace('동공산대', '동공 확대')



  x = x.replace('성교통', '성관계를 할 때 통증')#Para_Female_Child
  x = x.replace('성교', '성관계')
  x = x.replace('회음부', '성기')
  x = x.replace('외음부', '성기')
  x = x.replace('초경(-)', '초경 안함')
  x = x.replace('초경 전', '초경 안함')

  x = x.replace('오심', '구역감')#digestive(jaundice)
  x = x.replace('소양증', '가려움증')
  x = x.replace('쓰리다', '속쓰림')#hematemesis(digestive)
  return x

In [ ]:
# for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
#  data[m] = data.apply(lambda x : replace_diffi_to_easy(x[m]) , axis = 1 )

for i in tqdm(range(len(data))):
  data['All'][i] = replace_diffi_to_easy(data['All'][i])

  0%|          | 0/2215 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   level5/diagnosis  2215 non-null   object 
 1   Chief complaint   2215 non-null   object 
 2   Sex               2215 non-null   object 
 3   Age               2215 non-null   object 
 4   Height            2215 non-null   int64  
 5   Weight            2215 non-null   int64  
 6   Onset             2215 non-null   object 
 7   Location          2215 non-null   object 
 8   Duration          2215 non-null   object 
 9   Course            2215 non-null   object 
 10  Experience        2215 non-null   object 
 11  Character         2215 non-null   object 
 12  Associated Sx.    2215 non-null   object 
 13  Factor            2215 non-null   object 
 14  Event             2215 non-null   object 
 15  약물 투약력            2215 non-null   object 
 16  사회력               2215 non-null   object 


## 12) Erase stopwords using konlpy

In [ ]:
!pip install konlpy
import konlpy.tag
okt = konlpy.tag.Okt() # 객체 생성

In [ ]:
stopwords = ['질문', '문의', '관련', '그대로', '계속', '답변', '선생님', '관련문의',
              '한지', '자주', '좀', '쪽', '자꾸', '요즘', '몇개', '무조건', '하나요',
              '안해', '경우', '최근', '및', '몇', '달', '일반', '전날', '저번',
              '말', '일어나지', '며칠', '먹기', '지난번', '글', '때문', '너', '무',
              '시', '잔', '뒤', '지속', '막', '것', '이건', '뭔가', '다시', '그',
              '무슨', '안', '난', '기', '후', '거리', '뭘', '저', '뭐', '답젼',
              '평생', '회복', '반', '감사', '의사', '보험', '학생', '제발', '살짝',
              '느낌', '제', '대해','문제', '전','정도', '왜', '거', '가요',
              '의심', '추천', '를', '지금', '무엇', '관해', '리', '세',
              '로', '목적', '그냥', '거의', '고민', '다음', '이틀', '항상', '뭐', '때',
              '요',  '이후', '혹시', '안녕하세요',
              '안녕','선생','끼','일','식','첨부','말씀','이번','분','년','진단','밥',
              '속','년','동안','코딩','바','평소','게','주','올해','월','외','소견','오후','병원',
              '어머니','군데','여러분','전문가','건','아버지','주일','센티','동안','건가요',
              '의견','건강','세일','결까요','학원','수업','밤','부모','적','가족','대학생',
              '무언가','이게','무엇','포함','살','사진','제','가능','중','기재','아이',
              '저녁','안심','걱정','씨','며칠','동네','어디','하루','동생','해외','얘',
              '학년','사람','직장인','나이','키','몸무게','엄마','부탁','해석','혹','시가'
             '의', '가', '이', '은', '들', '는', '잘', '걍', '과', '도', '을'
             '를', '으로', '자', '에', '와', '하다', '다', '.', ',']

In [ ]:
def erase_stopwords(text):
  X_train =[]
  temp_x = okt.morphs(text, stem=True)
  #print(temp_x)
  temp_x = [word for word in temp_x if not word in stopwords]
  temp_x = re.findall(r'\w+', str(temp_x))
  X_train.append(temp_x)
  #print(X_train)
  temp_x = ' '.join(map(str, X_train))
  
  return ' '.join(re.findall(r'\w+', str(temp_x)))

In [ ]:
#for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
#  data[m] = data.apply(lambda x : erase_stopwords(x[m]) , axis = 1 )

#data['Chief complaint'] = data.apply(lambda x : erase_stopwords(x['Chief complaint']) , axis = 1 )

for i in tqdm(range(len(data))):
  data['All'][i] = erase_stopwords(data['All'][i])

  0%|          | 0/2215 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
#data.to_csv('processed_DB_5.17.csv', encoding='utf-8-sig')

In [ ]:
#import pandas as pd

#data = pd.read_csv('processed_DB_5.17.csv')
#data.drop('Unnamed: 0', axis=1, inplace=True)
#data.fillna('', inplace=True)
#data

# 2. Select features
https://datascience.stackexchange.com/questions/74688/how-to-use-multiple-text-features-for-nlp-classifier

In [ ]:
women_data = data.copy()

In [ ]:
men_data = data[data['Level1'] != 'para']
men_data = men_data[men_data['level5/diagnosis'] != '폐경증후군']
men_data = men_data[men_data['level5/diagnosis'] != '폐경에 의한 고혈압']

In [ ]:
print(len(women_data))
print(len(men_data))

2215
2000


In [ ]:
women_train = women_data.copy()
men_train = men_data.copy()

In [ ]:
from io import StringIO
col = ['level5/diagnosis', 'All']#, 'Height', 'Weight']
women_df = women_train[col]
men_df = men_train[col]

women_df = women_df[pd.notnull(women_df['All'])]
men_df = men_df[pd.notnull(men_df['All'])]

#df['category_id'] = df['level5/diagnosis'].factorize()[0]
#category_id_df = df[['level5/diagnosis', 'category_id']].drop_duplicates().sort_values('category_id')
#category_to_id = dict(category_id_df.values)
#id_to_category = dict(category_id_df[['category_id', 'level5/diagnosis']].values)

men_df.head()

,level5/diagnosis,All
0,소화성 궤양,배 아프다 50 대다 남자 1일 명치 부위 심해지다 이전 에도 3 차례 통증 보다 ...
1,소화성 궤양,갑자기 배 너무 아프다 30 대다 남자 오늘 새벽 명치 에서 시작 현재 복부 전체 ...
2,소화성 궤양,속이다 쓰리다 70 대다 여자 1 명치 부위 심해지다 속이다 쓰리다 약하다 구역질 ...
3,소화성 궤양,속이다 쓰리다 10 대다 남자 2 명치 부위 심해지다 명치 부위 누르다 아픔 쓰리다...
4,소화성 궤양,배 아프다 60 대다 남자 1시간 명치 부위 심해지다 명치 부위 아프다 쓰리다 고혈...


In [ ]:
#df.to_csv('digestive.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split

w_ori_X_train, w_ori_X_test, w_ori_y_train, w_ori_y_test = train_test_split(women_df, women_df['level5/diagnosis'], random_state = 0)
m_ori_X_train, m_ori_X_test, m_ori_y_train, m_ori_y_test = train_test_split(men_df, men_df['level5/diagnosis'], random_state = 0)

In [ ]:
#!pip3 install tensorflow_addons

# Tokenizer: LMKor

In [ ]:
#!pip install kobert_transformers
#!pip install sentencepiece
!pip install transformers

In [ ]:
# funnel: https://github.com/kiyoungkim1/LMkor
from transformers import FunnelTokenizerFast, FunnelModel
tokenizer_funnel = FunnelTokenizerFast.from_pretrained("kykim/funnel-kor-base")
tokenizer_funnel

PreTrainedTokenizerFast(name_or_path='kykim/funnel-kor-base', vocab_size=42000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<sep>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>'})

In [ ]:
#!pip install konlpy
#import konlpy.tag
#okt = konlpy.tag.Okt() # 객체 생성

#stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '을'
#             '를', '으로', '자', '에', '와', '한', '하다', '요']

In [ ]:
def only_letters_num(x):
  #x = okt.morphs(x, stem=True)
  #x = [word for word in x if not word in stopwords]
  return ' '.join(re.findall(r'\w+', str(x)))

In [ ]:
import re

w_ori_X_train2 = w_ori_X_train.copy()
w_ori_X_test2 = w_ori_X_test.copy()

w_ori_X_train2['All'] = w_ori_X_train.apply(lambda x : only_letters_num(x['All']) , axis = 1 )
w_ori_X_test2['All'] = w_ori_X_test.apply(lambda x : only_letters_num(x['All']) , axis = 1 )

w_document_bert_train = ["[CLS] " + str(s) + " [SEP]" for s in w_ori_X_train2['All']]
w_document_bert_test = ["[CLS] " + str(s) + " [SEP]" for s in w_ori_X_test2['All']]

w_ko_tokenized_texts_train = [tokenizer_funnel.tokenize(s) for s in w_document_bert_train]
w_ko_tokenized_texts_test = [tokenizer_funnel.tokenize(s) for s in w_document_bert_test]

##############
m_ori_X_train2 = m_ori_X_train.copy()
m_ori_X_test2 = m_ori_X_test.copy()

m_ori_X_train2['All'] = m_ori_X_train.apply(lambda x : only_letters_num(x['All']) , axis = 1 )
m_ori_X_test2['All'] = m_ori_X_test.apply(lambda x : only_letters_num(x['All']) , axis = 1 )

m_document_bert_train = ["[CLS] " + str(s) + " [SEP]" for s in m_ori_X_train2['All']]
m_document_bert_test = ["[CLS] " + str(s) + " [SEP]" for s in m_ori_X_test2['All']]

m_ko_tokenized_texts_train = [tokenizer_funnel.tokenize(s) for s in m_document_bert_train]
m_ko_tokenized_texts_test = [tokenizer_funnel.tokenize(s) for s in m_document_bert_test]

In [ ]:
print(w_ori_X_train['All'][0])
print(w_ori_X_train2['All'][0])

배 아프다 50 대다 남자 1일 명치 부위 심해지다 이전 에도 3 차례 통증 보다 약하다 칼 찢기다 듯 한 통증 심하다 퍼지다 통증 등 퍼지다 구토 쓰리다 응급 어지러움 갈증 소 변량 감소 자세 따르다 변화 오른쪽 돌 아누 우 면 완화 반듯이 누우 면 악화 술 1 6 7 번 소주 2 3 병 담배 30 갑년 식사 매우 불규칙 직업 포클레인 조종수 과체중
배 아프다 50 대다 남자 1일 명치 부위 심해지다 이전 에도 3 차례 통증 보다 약하다 칼 찢기다 듯 한 통증 심하다 퍼지다 통증 등 퍼지다 구토 쓰리다 응급 어지러움 갈증 소 변량 감소 자세 따르다 변화 오른쪽 돌 아누 우 면 완화 반듯이 누우 면 악화 술 1 6 7 번 소주 2 3 병 담배 30 갑년 식사 매우 불규칙 직업 포클레인 조종수 과체중


In [ ]:
print(max(len(l) for l in w_ko_tokenized_texts_train))
print(max(len(l) for l in m_ko_tokenized_texts_train))

140
140


In [ ]:
from keras.preprocessing.text import Tokenizer

w_max_words = 1800 #Choose only 1800 words which has high frequency
w_tokenizer = Tokenizer(num_words = w_max_words)
w_tokenizer.fit_on_texts(w_ko_tokenized_texts_train)

w_X_train_sequence = w_tokenizer.texts_to_sequences(w_ko_tokenized_texts_train)
w_X_test_sequence = w_tokenizer.texts_to_sequences(w_ko_tokenized_texts_test)

w_y_train_dummies = pd.get_dummies(women_df['level5/diagnosis']).loc[w_ori_X_train.index]
w_y_test_dummies = pd.get_dummies(women_df['level5/diagnosis']).loc[w_ori_X_test.index]

from keras.preprocessing.sequence import pad_sequences
w_max_len = max(len(l) for l in w_ko_tokenized_texts_train)
w_X_train_sequence = pad_sequences(w_X_train_sequence, maxlen=w_max_len)
w_X_test_sequence = pad_sequences(w_X_test_sequence, maxlen=w_max_len)

###############
m_max_words = 1600
m_tokenizer = Tokenizer(num_words = m_max_words)
m_tokenizer.fit_on_texts(m_ko_tokenized_texts_train)

m_X_train_sequence = m_tokenizer.texts_to_sequences(m_ko_tokenized_texts_train)
m_X_test_sequence = m_tokenizer.texts_to_sequences(m_ko_tokenized_texts_test)

m_y_train_dummies = pd.get_dummies(men_df['level5/diagnosis']).loc[m_ori_X_train.index]
m_y_test_dummies = pd.get_dummies(men_df['level5/diagnosis']).loc[m_ori_X_test.index]

m_max_len = max(len(l) for l in m_ko_tokenized_texts_train)
m_X_train_sequence = pad_sequences(m_X_train_sequence, maxlen=m_max_len)
m_X_test_sequence = pad_sequences(m_X_test_sequence, maxlen=m_max_len)

In [ ]:
print(len(w_y_train_dummies.columns))
print(len(m_y_test_dummies.columns))

225
193


In [ ]:
print(len(w_tokenizer.index_word) + 1)
print(len(w_tokenizer.word_counts))

print(len(m_tokenizer.index_word) + 1)
print(len(m_tokenizer.word_counts))

1765
1764
1606
1605


In [ ]:
print(w_tokenizer.num_words)
print(m_tokenizer.num_words)

1800
1600


In [ ]:
print(w_max_len)
print(m_max_len)

140
140


In [ ]:
print(len(women_df['level5/diagnosis'].unique()))
print(len(men_df['level5/diagnosis'].unique()))

225
193


In [ ]:
#women_df.to_csv("women_all_processed.csv",index=False)

#men_df.to_csv("women_all_processed.csv",index=False)

In [ ]:
#women_df=women_df.to_csv('/content/gdrive/MyDrive/캡스톤/coco/women_all_processed.csv')
#men_df=men_df.to_csv('/content/gdrive/MyDrive/캡스톤/coco/men_all_processed.csv')

# class weight

In [ ]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(class_weight ='balanced',
                                                 classes = np.unique(men_df['level5/diagnosis']),
                                                 y = men_df['level5/diagnosis'])

m_class_weights = dict(enumerate(class_weights))

In [ ]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(class_weight ='balanced',
                                                 classes = np.unique(women_df['level5/diagnosis']),
                                                 y = women_df['level5/diagnosis'])

w_class_weights = dict(enumerate(class_weights))

# 여기서부터 모델 돌리면 된다.!!

In [ ]:
import pandas as pd
women_df=pd.read_csv('/content/gdrive/MyDrive/캡스톤/coco/women_all_processed.csv')
men_df=pd.read_csv('/content/gdrive/MyDrive/캡스톤/coco/men_all_processed.csv')

# Model

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

## Multi (LSTM + Conv) _W

In [ ]:
def w_top_k_hitrate(predicted_proba, n=2):

  eval_metric = []
  top_k_result = tf.math.top_k(predicted_proba, k=n, sorted=True)
  for i in range(len(top_k_result[1])):
    first = top_k_result[1][i][0]
    second = top_k_result[1][i][1]
    third = top_k_result[1][i][2]

    if w_ori_X_test['level5/diagnosis'].values[i] in (w_y_test_dummies.columns[first],
                                                      w_y_test_dummies.columns[second],
                                                      w_y_test_dummies.columns[third]):
      eval_metric.append(1)
    else:
      eval_metric.append(0)

  return np.mean(eval_metric)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool1D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D, BatchNormalization, SimpleRNN, MaxPooling1D
from keras.models import Model
from keras.layers import Embedding, Dense, LSTM, GlobalMaxPooling1D, Dropout, Conv1D, Bidirectional
from keras.models import Sequential, load_model, save_model
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras as keras

w_VOCAB_SIZE = len(w_tokenizer.index_word) + 1
num_filters = 512
filter_sizes = [5, 7, 9]




from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, Dropout, Concatenate

embedding_dim = 1024

# LSTM + Attention model
inputs = Input(shape=(w_max_len,), dtype='int32')
embedding = Embedding(w_VOCAB_SIZE, embedding_dim)(inputs) 

lstm = Bidirectional(LSTM(512, dropout=0.25, return_sequences = True))(embedding)
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(512, dropout=0.25, return_sequences=True, return_state=True))(lstm)
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

## add attention layers
attention = BahdanauAttention(256) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

x = Dense(2400, activation='relu')(context_vector)
x = Dropout(0.3)(x)
x = Dense(800, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(units=len(women_df['level5/diagnosis'].unique()), activation='softmax')(x)



'''
lstm_1 = Bidirectional(LSTM(512, dropout=0.25, return_sequences = True))(embedding)
lstm_2 = Bidirectional(LSTM(512, dropout=0.25, return_sequences = True))(embedding)
lstm_3 = Bidirectional(LSTM(512, dropout=0.25, return_sequences = True))(embedding)

lstm_maxpool_0 = MaxPooling1D(pool_size=4)(lstm_1)
lstm_maxpool_1 = MaxPooling1D(pool_size=4)(lstm_2)
lstm_maxpool_2 = MaxPooling1D(pool_size=4)(lstm_3)

n_conv_0 = Conv1D(num_filters, kernel_size=(filter_sizes[0]),
                padding='valid', activation='relu')(lstm_maxpool_0)
n_conv_1 = Conv1D(num_filters, kernel_size=(filter_sizes[1]),
                padding='valid', activation='relu')(lstm_maxpool_1)
n_conv_2 = Conv1D(num_filters, kernel_size=(filter_sizes[2]),
                padding='valid', activation='relu')(lstm_maxpool_2)

#MaxPool1D
n_maxpool_0 = GlobalMaxPooling1D()(n_conv_0)
n_maxpool_1 = GlobalMaxPooling1D()(n_conv_1)
n_maxpool_2 = GlobalMaxPooling1D()(n_conv_2)

##############
conv_0 = Conv1D(num_filters, kernel_size=(filter_sizes[0]),
                padding='valid', activation='relu')(embedding)
conv_1 = Conv1D(num_filters, kernel_size=(filter_sizes[1]),
                padding='valid', activation='relu')(embedding)
conv_2 = Conv1D(num_filters, kernel_size=(filter_sizes[2]),
                padding='valid', activation='relu')(embedding)

#MaxPool1D
maxpool_0 = GlobalMaxPooling1D()(conv_0)
maxpool_1 = GlobalMaxPooling1D()(conv_1)
maxpool_2 = GlobalMaxPooling1D()(conv_2)
################

concatenated = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2,
                                    n_maxpool_0, n_maxpool_1, n_maxpool_2])
flatten_layer = Flatten()(concatenated)
dropout_layer1 = Dropout(0.5)(flatten_layer)

dense_layer3 = Dense(units=2400, activation='relu')(dropout_layer1)
dropout_layer4 = Dropout(0.5)(dense_layer3)

dense_layer4 = Dense(units=800, activation='relu')(dropout_layer4)
dropout_layer5 = Dropout(0.5)(dense_layer4)

output = Dense(units=len(women_df['level5/diagnosis'].unique()),
               activation='softmax')(dropout_layer5)
'''

w_model = Model(inputs=inputs, outputs=output)

w_model.compile(optimizer='adam', loss='categorical_crossentropy',
                metrics=['accuracy',
                         tf.keras.metrics.Precision(name='precision')])



In [ ]:
women_df

,Unnamed: 0,level5/diagnosis,All
0,0,소화성 궤양,배 아프다 50 대다 남자 1일 명치 부위 심해지다 이전 에도 3 차례 통증 보다 ...
1,1,소화성 궤양,갑자기 배 너무 아프다 30 대다 남자 오늘 새벽 명치 에서 시작 현재 복부 전체 ...
2,2,소화성 궤양,속이다 쓰리다 70 대다 여자 1 명치 부위 심해지다 속이다 쓰리다 약하다 구역질 ...
3,3,소화성 궤양,속이다 쓰리다 10 대다 남자 2 명치 부위 심해지다 명치 부위 누르다 아픔 쓰리다...
4,4,소화성 궤양,배 아프다 60 대다 남자 1시간 명치 부위 심해지다 명치 부위 아프다 쓰리다 고혈...
...,...,...,...
2210,2210,갑상샘저하증,콜레스테롤 높다 대요 50 대다 남자 1 개월 처음 피곤 식욕 저하 체중 증가 추위...
2211,2211,쿠싱증후군,콜레스테롤 높다 대요 30 대다 여자 1 처음 체중 변화 1 개월 근력 저하 복부 ...
2212,2212,폐경증후군,콜레스테롤 높다 대요 60 대다 여자 1 개월 처음 습관 운동 함 12년 폐경 과체중
2213,2213,폐경증후군,콜레스테롤 높다 대요 60 대다 여자 3 개월 처음 10년 폐경 정상


In [ ]:
history = w_model.fit(w_X_train_sequence, w_y_train_dummies, epochs=100, 
                      batch_size = 512, class_weight = w_class_weights,
                      validation_split=0.2)

Epoch 1/100
3/3 [==============================] - 19s 4s/step - loss: 5.4234 - accuracy: 0.0023 - precision: 0.0000e+00 - val_loss: 5.4197 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 9s 3s/step - loss: 5.4176 - accuracy: 0.0030 - precision: 0.0000e+00 - val_loss: 5.4195 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 9s 3s/step - loss: 5.4139 - accuracy: 0.0023 - precision: 0.0000e+00 - val_loss: 5.4330 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 9s 3s/step - loss: 5.3743 - accuracy: 0.0030 - precision: 0.0000e+00 - val_loss: 5.4470 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 9s 3s/step - loss: 5.3164 - accuracy: 0.0045 - precision: 0.0000e+00 - val_loss: 5.4490 - val_accuracy: 0.0060 - val_precision: 0.0000e+00
Epoch 6/100
3/3 [================

In [ ]:
test_result = w_model.evaluate(w_X_test_sequence, w_y_test_dummies)

print("Accuracy of Test: {:.4f}".format(test_result[1])) #(TP+TN)/(TP+FN+FP+TN)
print("Precision of Test: {:.4f}".format(test_result[2])) #(TP/(TP+FP))
#print("F1-score of Test: {:.4f}".format(np.mean(test_result[3])))
print("top_k_result: {:.4f}".format(w_top_k_hitrate(w_model.predict(w_X_test_sequence), n=3)))

18/18 [==============================] - 3s 137ms/step - loss: 3.5353 - accuracy: 0.6534 - precision: 0.6883
Accuracy of Test: 0.6534
Precision of Test: 0.6883
top_k_result: 0.7437


In [ ]:
#dropout 0.3
#Accuracy of Test: 0.8195
#Precision of Test: 0.9523
#top_k_result: 0.9152

#Accuracy of Test: 0.8649
#Precision of Test: 0.8894
#top_k_result: 0.9272

In [ ]:
top_k_result = tf.math.top_k(w_model.predict(w_X_test_sequence), k=1, sorted=True)
first = top_k_result[1]
b = w_y_test_dummies.columns[first]

w_ori_y_test = np.asarray(w_ori_y_test)

compare_result = pd.concat([pd.DataFrame(w_ori_y_test, columns=['True']).reset_index(drop=True),
                            pd.DataFrame(b, columns=['predict']).reset_index(drop=True)],
                           axis=1)

misclassified = np.where(compare_result['True'] != compare_result['predict'])
compare_result.iloc[misclassified]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,True,predict
0,갑상샘저하증,백의 고혈압
1,역류성 식도염,소화성 궤양
2,기흉,알레르기성 비염
4,급성 충수염,요로결석
5,대상포진,대동맥 박리
...,...,...
543,폐렴,상기도출혈
546,약물 / 알콜에 의한 증상,신경인성 방광
547,유방낭종,급성 유방염
548,협착증,소화성 궤양


## Multi (LSTM -> Conv) _M

In [ ]:
def m_top_k_hitrate(predicted_proba, n=2):

  eval_metric = []
  top_k_result = tf.math.top_k(predicted_proba, k=n, sorted=True)
  for i in range(len(top_k_result[1])):
    first = top_k_result[1][i][0]
    second = top_k_result[1][i][1]
    third = top_k_result[1][i][2]

    if m_ori_X_test['level5/diagnosis'].values[i] in (m_y_test_dummies.columns[first],
                                                      m_y_test_dummies.columns[second],
                                                      m_y_test_dummies.columns[third]):
      eval_metric.append(1)
    else:
      eval_metric.append(0)

  return np.mean(eval_metric)

In [ ]:
from keras.layers import MaxPooling1D
m_VOCAB_SIZE = len(m_tokenizer.index_word) + 1
num_filters = 512
filter_sizes = [5, 7, 9]



from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, Dropout, Concatenate



# LSTM + Attention model

#embedding = Embedding(w_VOCAB_SIZE, embedding_dim)(inputs) 
embedding_dim = 1024
inputs = Input(shape=(m_max_len,), dtype='int32')
embedding = Embedding(m_VOCAB_SIZE, 1000)(inputs) #800하면 top_k가 0.01높지만 나머지는 0.01낮다.
lstm = Bidirectional(LSTM(512, dropout=0.25, return_sequences = True))(embedding)
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(512, dropout=0.25, return_sequences=True, return_state=True))(lstm)
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

## add attention layers
attention = BahdanauAttention(256) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

x = Dense(2400, activation='relu')(context_vector)
x = Dropout(0.5)(x)
x = Dense(800, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(units=len(men_df['level5/diagnosis'].unique()), activation='softmax')(x)

'''#lstm_1 = LSTM(512, dropout=0.2, return_sequences=True)(embedding)
#lstm_2 = LSTM(512, dropout=0.2, return_sequences=True)(embedding)
#lstm_3 = LSTM(512, dropout=0.2, return_sequences=True)(embedding)

lstm_1 = LSTM(512, dropout=0.3, return_sequences=True)(embedding)
lstm_2 = LSTM(512, dropout=0.3, return_sequences=True)(embedding)
lstm_3 = LSTM(512, dropout=0.3, return_sequences=True)(embedding)

lstm_maxpool_0 = MaxPooling1D(pool_size=4)(lstm_1)
lstm_maxpool_1 = MaxPooling1D(pool_size=4)(lstm_2)
lstm_maxpool_2 = MaxPooling1D(pool_size=4)(lstm_3)

n_conv_0 = Conv1D(num_filters, kernel_size=(filter_sizes[0]),
                padding='valid', activation='relu')(lstm_maxpool_0)
n_conv_1 = Conv1D(num_filters, kernel_size=(filter_sizes[1]),
                padding='valid', activation='relu')(lstm_maxpool_1)
n_conv_2 = Conv1D(num_filters, kernel_size=(filter_sizes[2]),
                padding='valid', activation='relu')(lstm_maxpool_2)

#MaxPool1D
n_maxpool_0 = GlobalMaxPooling1D()(n_conv_0)
n_maxpool_1 = GlobalMaxPooling1D()(n_conv_1)
n_maxpool_2 = GlobalMaxPooling1D()(n_conv_2)

##############
conv_0 = Conv1D(num_filters, kernel_size=(filter_sizes[0]),
                padding='valid', activation='relu')(embedding)
conv_1 = Conv1D(num_filters, kernel_size=(filter_sizes[1]),
                padding='valid', activation='relu')(embedding)
conv_2 = Conv1D(num_filters, kernel_size=(filter_sizes[2]),
                padding='valid', activation='relu')(embedding)

#MaxPool1D
maxpool_0 = GlobalMaxPooling1D()(conv_0)
maxpool_1 = GlobalMaxPooling1D()(conv_1)
maxpool_2 = GlobalMaxPooling1D()(conv_2)
################

concatenated = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2,
                                    n_maxpool_0, n_maxpool_1, n_maxpool_2])
flatten_layer = Flatten()(concatenated)
dropout_layer1 = Dropout(0.5)(flatten_layer)

dense_layer3 = Dense(units=2400, activation='relu')(dropout_layer1)
dropout_layer4 = Dropout(0.5)(dense_layer3)

dense_layer4 = Dense(units=800, activation='relu')(dropout_layer4)
dropout_layer5 = Dropout(0.5)(dense_layer4)

output = Dense(units=len(men_df['level5/diagnosis'].unique()),
               activation='softmax')(dropout_layer5)'''

m_model = Model(inputs=inputs, outputs=output)

m_model.compile(optimizer='adam', loss='categorical_crossentropy',
                metrics=['accuracy',
                         tf.keras.metrics.Precision(name='precision')])



In [ ]:
history = m_model.fit(m_X_train_sequence, m_y_train_dummies, epochs=100,
                      batch_size = 128, class_weight = m_class_weights,
                      validation_split=0.2)

Epoch 1/100
10/10 [==============================] - 19s 1s/step - loss: 5.2769 - accuracy: 0.0075 - precision: 0.0000e+00 - val_loss: 5.2591 - val_accuracy: 0.0033 - val_precision: 0.0000e+00
Epoch 2/100
10/10 [==============================] - 9s 955ms/step - loss: 5.2616 - accuracy: 0.0100 - precision: 0.0000e+00 - val_loss: 5.2441 - val_accuracy: 0.0033 - val_precision: 0.0000e+00
Epoch 3/100
10/10 [==============================] - 9s 949ms/step - loss: 5.2742 - accuracy: 0.0067 - precision: 0.0000e+00 - val_loss: 5.3276 - val_accuracy: 0.0033 - val_precision: 0.0000e+00
Epoch 4/100
10/10 [==============================] - 9s 946ms/step - loss: 5.0943 - accuracy: 0.0133 - precision: 1.0000 - val_loss: 5.1758 - val_accuracy: 0.0033 - val_precision: 0.0000e+00
Epoch 5/100
10/10 [==============================] - 9s 947ms/step - loss: 5.0070 - accuracy: 0.0117 - precision: 0.0000e+00 - val_loss: 5.1298 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 6/100
10/10 [========

In [ ]:
test_result = m_model.evaluate(m_X_test_sequence, m_y_test_dummies)

print("Accuracy of Test: {:.4f}".format(test_result[1])) #(TP+TN)/(TP+FN+FP+TN)
print("Precision of Test: {:.4f}".format(test_result[2])) #(TP/(TP+FP))
#print("F1-score of Test: {:.4f}".format(np.mean(test_result[3])))
print("top_k_result: {:.4f}".format(m_top_k_hitrate(m_model.predict(m_X_test_sequence), n=3)))

16/16 [==============================] - 2s 138ms/step - loss: 2.7229 - accuracy: 0.7380 - precision: 0.7745
Accuracy of Test: 0.7380
Precision of Test: 0.7745
top_k_result: 0.8060


In [ ]:
'''
women
Accuracy of Test: 0.6390
Precision of Test: 0.7478
top_k_result: 0.7238

men
Accuracy of Test: 0.6860
Precision of Test: 0.8076
top_k_result: 0.7880

'''

'\nwomen\nAccuracy of Test: 0.6390\nPrecision of Test: 0.7478\ntop_k_result: 0.7238\n\nmen\nAccuracy of Test: 0.6860\nPrecision of Test: 0.8076\ntop_k_result: 0.7880\n\n'

In [ ]:
top_k_result = tf.math.top_k(m_model.predict(m_X_test_sequence), k=1, sorted=True)
first = top_k_result[1]
b = m_y_test_dummies.columns[first]

m_ori_y_test = np.asarray(m_ori_y_test)

compare_result = pd.concat([pd.DataFrame(m_ori_y_test, columns=['True']).reset_index(drop=True),
                            pd.DataFrame(b, columns=['predict']).reset_index(drop=True)],
                           axis=1)

misclassified = np.where(compare_result['True'] != compare_result['predict'])
compare_result.iloc[misclassified]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,True,predict
8,용혈성 요독증후군,부정맥
9,급성 췌장염,급성 충수염
13,소아성 용종,결장암 / 직장암
20,횡문근융해증,요로 감염
24,크롬세포친화종,약물 유발 빈맥
...,...,...
482,기립성 저혈압(혈량저하증 동반 가능),용혈병
485,난소 염전,자궁외임신
492,역류성 식도염,절박성 요실금
493,만성 신부전,악성 고혈암


# Save Models

In [ ]:
import pickle

# Save processed disease names
with open("/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_men_diseases_v9.txt", "wb") as fp:
  pickle.dump(m_y_test_dummies.columns, fp)

with open("/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_women_diseases_v9.txt", "wb") as fp:
  pickle.dump(w_y_test_dummies.columns, fp)

# Save tokenizer
with open('/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_men_tokenizer_v9.pickle', 'wb') as handle:
    pickle.dump(m_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_women_tokenizer_v9.pickle', 'wb') as handle:
    pickle.dump(w_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# Save ML model
#with open('ml_random_forest.pkl', 'wb') as file:
#    pickle.dump(ml_model, file)

# Save DL model
## 1
#Input(shape=(m_max_len,), dtype='int32')
#m_model.add(tf.keras.layers.Flatten(input_shape=(m_max_len,)))
#w_model.add(tf.keras.layers.Flatten(input_shape=(w_max_len,)))
#m_model.save('/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_men_model.h5')

#w_model.save('/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_women_model.h5')

with open("/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_men_model_v9.h5", "wb") as fp:
  pickle.dump(m_model, fp)

with open("/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_women_model_v9.h5", "wb") as fp:
  pickle.dump(w_model, fp)
#m_model.save('/content/gdrive/MyDrive/캡스톤/Disease_models/ver2/LSTMAttention_men_model.h5')
#w_model.save('/content/gdrive/MyDrive/캡스톤/Disease_models/ver2/LSTMAttention_women_model.h5')

## 2
#model_json = model.to_json()
#with open("baseline_model.json", "w") as json_file : 
#    json_file.write(model_json)

#model.save_weights("baseline_model_weights.h5")

INFO:tensorflow:Assets written to: ram://6d7a3bf7-490d-41a7-bd57-1f08be8c4ff9/assets


INFO:tensorflow:Assets written to: ram://6d7a3bf7-490d-41a7-bd57-1f08be8c4ff9/assets


INFO:tensorflow:Assets written to: ram://8b77f2de-2238-4558-97a2-951ee3a1a750/assets


INFO:tensorflow:Assets written to: ram://8b77f2de-2238-4558-97a2-951ee3a1a750/assets


In [ ]:
w_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 140)]        0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 140, 1024)    1807360     ['input_7[0][0]']                
                                                                                                  
 bidirectional_12 (Bidirectiona  (None, 140, 1024)   6295552     ['embedding_6[0][0]']            
 l)                                                                                               
                                                                                                  
 bidirectional_13 (Bidirectiona  [(None, 140, 1024),  6295552    ['bidirectional_12[0][0]'] 

# For test

In [ ]:
!pip install transformers
!pip install konlpy
from transformers import FunnelTokenizerFast
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import pickle
import konlpy.tag
import re

In [ ]:
def to_nan(x):
  if(x == '-'):
    x = ''
  elif(x == '아니오'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '몰라요'):
    x = ''
  elif(x == '모릅니다'):
    x = ''
  elif(x == '모름'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '없습니다'):
    x = ''
  elif(x == '없어요'):
    x = ''
  elif(x == '없음'):
    x = ''
  elif(x == '.'):
    x = ''    
  return x

def NRS_to_text(text):

  a = re.findall('NRS.*?점', text) #NRS 5점
  if len(a) == 0:
    a = re.findall('NRS.*?\d~\d', text) #NRS 4~6 / NRS : 4~6
  if len(a) == 0:
    a = re.findall('NRS.*?\d', text) #NRS: 5
  
  try:
    t = re.findall('[0-9]+', a[0])

    if (len(t) == 2):
      score = (int(t[0])+int(t[1]))/ 2
    else:
      score = int(t[0])

    if score >= 7:
      to_text = '심함'
    elif score >= 4:
      to_text = '중간'
    else:
      to_text = '약함'

    text = text.replace(a[0], to_text)
  
  except:
    pass

  return text

from hanspell import spell_checker

def spelling_check(text):
  #print(text)
  result = spell_checker.check(text) 
  
  return result.as_dict()['checked'] 

def define_obesity(x):
  if x < 0:
    return '알 수 없음'
  elif x < 20.0:
    return '저체중'
  elif x <= 24.0:
    return '정상'
  elif x <= 29.0:
    return '과체중'
  else:
    return '비만'


def only_letters_num(x):
  return ' '.join(re.findall(r'\w+', str(x)))


okt = konlpy.tag.Okt() # 객체 생성
stopwords = ['질문', '문의', '관련', '그대로', '계속', '답변', '선생님', '관련문의',
              '한지', '자주', '좀', '쪽', '자꾸', '요즘', '몇개', '무조건', '하나요',
              '안해', '경우', '최근', '및', '몇', '달', '일반', '전날', '저번',
              '말', '일어나지', '며칠', '먹기', '지난번', '글', '때문', '너', '무',
              '시', '잔', '뒤', '지속', '막', '것', '이건', '뭔가', '다시', '그',
              '무슨', '안', '난', '기', '후', '거리', '뭘', '저', '뭐', '답젼',
              '평생', '회복', '반', '감사', '의사', '보험', '학생', '제발', '살짝',
              '느낌', '제', '대해','문제', '전','정도', '왜', '거', '가요',
              '의심', '추천', '를', '지금', '무엇', '관해', '리', '세',
              '로', '목적', '그냥', '거의', '고민', '다음', '이틀', '항상', '뭐', '때',
              '요',  '이후', '혹시', '안녕하세요',
              '안녕','선생','끼','일','식','첨부','말씀','이번','분','년','진단','밥',
              '속','년','동안','코딩','바','평소','게','주','올해','월','외','소견','오후','병원',
              '어머니','군데','여러분','전문가','건','아버지','주일','센티','동안','건가요',
              '의견','건강','세일','결까요','학원','수업','밤','부모','적','가족','대학생',
              '무언가','이게','무엇','포함','살','사진','제','가능','중','기재','아이',
              '저녁','안심','걱정','씨','며칠','동네','어디','하루','동생','해외','얘',
              '학년','사람','직장인','나이','키','몸무게','엄마','부탁','해석','혹','시가'
             '의', '가', '이', '은', '들', '는', '잘', '걍', '과', '도', '을'
             '를', '으로', '자', '에', '와', '하다', '다', '.', ',']
             
def erase_stopwords(text):
  temp_x = okt.morphs(text, stem=True)
  temp_x = [word for word in temp_x if not word in stopwords]
  temp_x = re.findall(r'\w+', str(temp_x))
  temp_x = ' '.join(map(str, temp_x))
  
  return ' '.join(re.findall(r'\w+', str(temp_x)))

In [ ]:
with open('/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_men_tokenizer_v9.pickle', 'rb') as handle:
    m_tokenizer = pickle.load(handle)

with open('/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_men_model_v9.h5', 'rb') as handle:
    m_loaded_mode = pickle.load(handle)

with open("/content/gdrive/MyDrive/캡스톤/coco/LSTMAttention_men_diseases_v9.txt", "rb") as fp:
    m_disease_codes = pickle.load(fp)

In [ ]:
test_df = {
  'Height': '180',
  'Weight': '75',
  'Age': '30대',
  'Chief complaint': '배가 아파요',
  'Sex': '남자',
  'Onset': '1일 전',
  'Location': '명치 부위',
  'Duration' : '지속',
  'Course': '심해짐',
  'Experience' : '이전에도 3차례, 통증은 이번보다 약했음',
  'Character': '칼로 찢기는 듯한 통증, NRS 8점, 방사통: 등으로 퍼짐',
  'Associated Sx.': '구토, 속쓰림, 어지러움, 갈증, 소변량 감소',
  'Factor': '오른쪽으로 돌아 누우면 완화',
  'Event': '',
  '약물 투약력': '',
  '사회력': '술: 1주일 6~7번, 식사는 매우 불규칙',
  '가족력': '',
  '외상력': '',
  '과거력': '',
  '여성력': ''
}

data = pd.DataFrame([test_df])

In [ ]:
# 두근거리다 50 대다 여자 1일 3시간 6 개월 부터 기적 규칙 빠르다 홍조 더위 견디다 못 불면 생리 량 감소 정상
test_df_w =  {
  'Height': '160',
  'Weight': '55',
  'Age': '50대',
  'Chief complaint': '두근거려요',
  'Sex': '여자',
  'Onset': '6개월 전',
  'Location': '아랫배 부위',
  'Duration' : '지속',
  'Course': '심해짐',
  'Experience' : '이전에도 3차례, 통증은 이번보다 약했음',
  'Character': ', NRS 2점',
  'Associated Sx.': '홍조, 더위를 못 견딤, 생리량 감소',
  'Factor': '',
  'Event': '',
  '약물 투약력': '',
  '사회력': '술: 1주일 2~3번, 식사는 규칙',
  '가족력': '',
  '외상력': '',
  '과거력': '갑상선암',
  '여성력': ''
}

data = pd.DataFrame([test_df_w])

In [ ]:
data.values

array([['180', '75', '30대', '배가 아파요', '남자', '1일 전', '명치 부위', '지속', '심해짐',
        '이전에도 3차례, 통증은 이번보다 약했음', '칼로 찢기는 듯한 통증, NRS 8점, 방사통: 등으로 퍼짐',
        '구토, 속쓰림, 어지러움, 갈증, 소변량 감소', '오른쪽으로 돌아 누우면 완화', '', '',
        '술: 1주일 6~7번, 식사는 매우 불규칙', '', '', '', '']], dtype=object)

In [ ]:
data

,Height,Weight,Age,Chief complaint,Sex,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력
0,180,75,30대,배가 아파요,남자,1일 전,명치 부위,지속,심해짐,"이전에도 3차례, 통증은 이번보다 약했음","칼로 찢기는 듯한 통증, NRS 8점, 방사통: 등으로 퍼짐","구토, 속쓰림, 어지러움, 갈증, 소변량 감소",오른쪽으로 돌아 누우면 완화,,,"술: 1주일 6~7번, 식사는 매우 불규칙",,,,


In [ ]:
# Handle Nan values 
data = data.fillna('-')

for i in range(len(data.columns)):
  data[data.columns[i]] = data.apply(lambda x : to_nan(x[data.columns[i]]) , axis = 1 )

# Define obesity by calculating BMI with Height and Weight
data['Height'] = data['Height'].replace('', '0')
data['Weight'] = data['Weight'].replace('', '0')

data['Height'] = data['Height'].astype('int')
data['Weight'] = data['Weight'].astype('int')

data['BMI'] = data['Weight'] / (data['Height']/100)**2
data['BMI'].fillna(-1, inplace=True)
data['Obesity'] = define_obesity(data['BMI'].values)
  
# Change Age to Groups of AgeX(-> Since we are going to receive their age as a group_format)
data['Age'] = data['Age'].astype(str)

In [ ]:
# Make a sentence
data['All'] = (data['Chief complaint'].values +'. '+ data['Age'].values + '. '+
                  data['Onset'].values + '. '+ data['Location'].values + '. ' +
                  data['Sex'].values + '. '+
                  data['Duration'].values + '. ' + data['Course'].values + '. ' +
                  data['Experience'].values + '. ' + data['Character'].values + '. ' +
                  data['Associated Sx.'].values+'. '+ data['Factor'].values + '. ' +
                  data['Event'].values + '. ' + data['약물 투약력'].values + '. ' +
                  data['사회력'].values + '. ' + data['가족력'].values + '. ' +
                  data['외상력'].values + '. ' + data['과거력'].values + '. ' +
                  data['여성력'].values + '. ' + data['Obesity'].values)

In [ ]:
data

,Height,Weight,Age,Chief complaint,Sex,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력,BMI,Obesity,All
0,180,75,30대,배가 아파요,남자,1일 전,명치 부위,지속,심해짐,"이전에도 3차례, 통증은 이번보다 약했음","칼로 찢기는 듯한 통증, NRS 8점, 방사통: 등으로 퍼짐","구토, 속쓰림, 어지러움, 갈증, 소변량 감소",오른쪽으로 돌아 누우면 완화,,,"술: 1주일 6~7번, 식사는 매우 불규칙",,,,,23.148148,정상,배가 아파요. 30대. 1일 전. 명치 부위. 남자. 지속. 심해짐. 이전에도 3차...


In [ ]:
len(data)

1

In [ ]:
# Change NRS to text
data['All'] = NRS_to_text(data['All'].values[0])

# Spelling Check
data['All'] = spelling_check(data['All'].values[0])

# Erase stopwords using konlpy
data['All'] = erase_stopwords(data['All'].values[0])

In [ ]:
data['All'] = only_letters_num(data['All'].values)
document_bert_data = ["[CLS] " + str(s) + " [SEP]" for s in data['All'].values]

In [ ]:
document_bert_data

['[CLS] 배 아프다 30 대다 1일 명치 부위 남자 심해지다 이전 에도 3 차례 통증 보다 약하다 칼 찢기다 듯 한 통증 심하다 방사 통 등 퍼지다 구토 쓰리다 어지러움 갈증 소 변량 감소 오른쪽 돌 아누 우 면 완화 술 1 6 7 번 식사 매우 불규칙 정상 [SEP]']

In [ ]:
m_max_len = 140
tokenizer_funnel = FunnelTokenizerFast.from_pretrained("kykim/funnel-kor-base")

if data['Sex'].values =='남자':
  ko_tokenized_texts_data = [tokenizer_funnel.tokenize(s) for s in document_bert_data]
  data_sequence = m_tokenizer.texts_to_sequences(ko_tokenized_texts_data)
  data_sequence = pad_sequences(data_sequence, maxlen = m_max_len).reshape(1,-1)

In [ ]:
y_prob = m_loaded_mode.predict(data_sequence)

In [ ]:
top_k_result = tf.math.top_k(y_prob, k=3, sorted=True)

first = top_k_result[1][0][0], top_k_result.values.numpy()[0][0]
second = top_k_result[1][0][1], top_k_result.values.numpy()[0][1]
third = top_k_result[1][0][2], top_k_result.values.numpy()[0][2]

first_pred_disease_name = m_disease_codes[first[0]]
second_pred_disease_name = m_disease_codes[second[0]]
third_pred_disease_name = m_disease_codes[third[0]]

In [ ]:
aa = pd.read_csv(DATA_IN_PATH + 'Disease_info.csv')
#aa.drop('Unnamed: 0', axis=1, inplace=True)
aa

,Unnamed: 0,원 질병이름,사이트 질병 이름,진료과,동의어,정의
0,0,ACEI(혈압강하제) 복용으로 인한 증상,(ACEI side effect),순환기내과,ACEI 부작용,혈압 강하제 중 ACEI 복용 후 나타나는 부작용을 의미합니다.
1,1,간염,간염(Hepatitis),소화기내과,"간세포 염증,간염증",간염은 간세포 조직에 염증이 생긴 것을 의미합니다.
2,2,간질성 폐질환,미만성 간질성 폐질환(Diffuse Interstitial lung disease),호흡기내과,"간질성 폐질환,미만성 침윤성 폐질환,특발성 간질성 폐렴,폐섬유증,폐섬유화증","미만성 간질성 폐질환(DILD)은 폐간질에 침범되는 비종양성, 비감염성 질환을 총칭..."
3,3,갈비연골염,갈비연골염,흉부외과,늑연골염,갈비연골염은 늑골과 흉골을 연결하는 갈비 연골에 염증이 생긴 것입니다.
4,4,감염성 관절염,감염성 관절염,류미티스과,관절염,감염성 관절염은 보통 박테리아나 경우에 따라 바이러스 또는 진균으로 인해 관절액과 ...
...,...,...,...,...,...,...
221,221,지주막하 출혈,뇌동맥류(Cerebral aneurysm),"영상의학과, 신경외과, 신경중재클리닉","뇌동맥꽈리,뇌지주막하 출혈,두개강 내 동맥류",뇌동맥류는 뇌동맥 일부가 약해져서 그 부분이 풍선이나 꽈리처럼 부풀어 오르는 질환을...
222,222,커피 / 약물에 의한 요붕증,요붕증(Diabetes insipidus),"내분비내과, 소아내분비대사과",항이뇨호르몬 결핍증,항이뇨호르몬 분비가 부족하거나 신장에서 제 기능을 하지 못하는 상태를 요붕증이라고 ...
223,223,터너증후군,터너 증후군 (Turner syndrome),"의학유전학센터, 소아내분비대사과, 의학유전학과",터너증후군,"터너 증후군은 여성의 키가 작고, 목이 짧고 두꺼우며, 성적 발달이 지연되는 유전 ..."
224,224,하지신경 손상,업데이트 예정 (전처리 중 drop 질병),업데이트 예정 (전처리 중 drop 질병),업데이트 예정 (전처리 중 drop 질병),업데이트 예정 (전처리 중 drop 질병)


# **ver9:**

embedding_dim = 1024, dense 2400 -- 800, dropout 0.5
stopwords 설정, epoch 100

In [ ]:
'''
women
Accuracy of Test: 0.6534
Precision of Test: 0.6883
top_k_result: 0.7437

men
Accuracy of Test: 0.7380
Precision of Test: 0.7745
top_k_result: 0.8060

'''

In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
print(first_pred_disease_name, round(first[1]*100, 2),'%','\n',
      '동의어:', first_info['동의어'].values[0]+'\n',
      '진료과: ', first_info['진료과'].values[0]+'\n',
      '질병 설명:', first_info['정의'].values[0]
      )

위암 85.26 % 
 동의어: Gastric cancer,Malignant neoplasm of stomach,cancer of the stomach,위 악성종양,위선암,위선암종,위암종
 진료과:  소화기내과, 종양내과, 방사선종양학과, 위장관외과, 암병원
 질병 설명: 위에 발생하는 악성 종양에는 위 선암, 림프종, 위 점막하 종양, 평활 근육종 등이 있습니다. 이 중에서 위 선암이 98%를 차지합니다. 따라서 위암은 일반적으로 위 선암을 말합니다. 위암은 위의 점막에서 발생합니다. 시간이 지남에 따라 점막하층, 근육층, 장막하층, 장막층으로 침윤합니다. 위암은 점막 또는 점막하층을 따라 위 내에 넓게 퍼지기도 하고, 점막층에서 장막층을 향해 깊이 퍼지기도 합니다. 위 주변의 임파선을 따라서, 혹은 혈류의 파급에 의해 간, 폐, 뼈 등의 여러 부위로 퍼질 수 있습니다.


In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
first_info['진료과'].values[0]

'소화기내과, 종양내과, 방사선종양학과, 위장관외과, 암병원'

In [ ]:
second_info = aa[aa['원 질병이름'] == second_pred_disease_name]
print(second_pred_disease_name, round(second[1]*100, 2),'%','\n',
      '동의어:', second_info['동의어'].values[0]+'\n',
      '진료과: ', second_info['진료과'].values[0]+'\n',
      '질병 설명:', second_info['정의'].values[0]
      )

단장 증후군 4.34 % 
 동의어: 작은창자증후군,짧은 창자증후군
 진료과:  소화기내과
 질병 설명: 단장 증후군은 소장이 짧아 영양소의 소화 흡수 기능이 저하됨으로써 발생하는 여러 가지 대사 이상을 의미합니다. 발병 유형에 따라 선천성과 후천성으로 구분합니다. 선천성은 태아가 짧은 소장을 갖고 태어나는 것을 의미합니다. 후천성은 다른 질병이나 외상으로 인해 잘라내고 남은 소장의 길이가 너무 짧아 장애를 일으키는 것을 의미합니다.


In [ ]:
third_info = aa[aa['원 질병이름'] == third_pred_disease_name]
print(third_pred_disease_name, round(third[1]*100, 2),'%','\n',
      '동의어:', third_info['동의어'].values[0]+'\n',
      '진료과: ', third_info['진료과'].values[0]+'\n',
      '질병 설명:', third_info['정의'].values[0]
      )

식도 정맥류 파열 4.05 % 
 동의어: Esophageal Varix
 진료과:  소화기내과
 질병 설명: 식도 정맥류는 식도에 있는 정맥이 혹처럼 부풀어 오르는 질환입니다. 이 질환은 간경변증의 주요 합병증입니다.


# **ver8:**

embedding_dim = 1024, dense 2048--512, 2400 -- 800, dropout 0.3, 0.5
stopwords 똑같이 만들어주기

In [ ]:
'''
women
Accuracy of Test: 0.6372
Precision of Test: 0.6879
top_k_result: 0.7491

men
Accuracy of Test: 0.6840
Precision of Test: 0.7534
top_k_result: 0.7860

'''

In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
print(first_pred_disease_name, round(first[1]*100, 2),'%','\n',
      '동의어:', first_info['동의어'].values[0]+'\n',
      '진료과: ', first_info['진료과'].values[0]+'\n',
      '질병 설명:', first_info['정의'].values[0]
      )

유당불내증 64.4 % 
 동의어: 우유 소화불량,유당분해효소결핍,유당불내증,젖당분해효소결핍,젖당불내성
 진료과:  가정의학과
 질병 설명: 유당은 우유에 함유된 물질입니다. 유당 분해 효소인 락타아제가 부족한 사람이 유당을 섭취하면, 유당이 소장에서 삼투 현상에 의해 수분을 끌어들임으로써 팽만감과 경련을 일으키고, 대장을 통과하면서 설사를 유발합니다. 이러한 현상을 유당분해효소결핍증이라 합니다.


In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
first_info['진료과'].values[0]

'가정의학과'

In [ ]:
second_info = aa[aa['원 질병이름'] == second_pred_disease_name]
print(second_pred_disease_name, round(second[1]*100, 2),'%','\n',
      '동의어:', second_info['동의어'].values[0]+'\n',
      '진료과: ', second_info['진료과'].values[0]+'\n',
      '질병 설명:', second_info['정의'].values[0]
      )

염증성 장질환 29.5 % 
 동의어: 국한성 창자염,염증성 장질환,육아종성 대장염,크론씨병
 진료과:  소화기내과, 대장항문외과, 소아소화기영양과
 질병 설명: 크론병은 입에서 항문까지 소화기관 전체에 발생할 수 있는 만성 염증성 장질환입니다. 궤양성 대장염과 달리 장의 모든 층에 염증이 침범할 수 있습니다. 주로 대장과 소장이 연결되는 부위인 회맹부에 발병하는 경우가 흔하고, 그 다음으로 대장, 회장 말단부, 소장 등에서 많이 발생합니다.


In [ ]:
third_info = aa[aa['원 질병이름'] == third_pred_disease_name]
print(third_pred_disease_name, round(third[1]*100, 2),'%','\n',
      '동의어:', third_info['동의어'].values[0]+'\n',
      '진료과: ', third_info['진료과'].values[0]+'\n',
      '질병 설명:', third_info['정의'].values[0]
      )

뇌척수액 비루 4.28 % 
 동의어: 업데이트 예정
 진료과:  신경외과
 질병 설명: 뇌척수액 비루란 뇌척수액이 마치 콧물처럼 코로 새어나오는 질환입니다. 맑고 물 같은 콧물이 계속 나오며, 아랫배에 힘을 주었을 때 더 많이 나온다면 뇌척수액비루일 가능성이 있습니다.


# **ver7:**

embedding_dim = 1024, dense 2400 -- 800, dropout 0.25, 0.5
stopwords 똑같이 만들어주기

In [ ]:
'''
women
Accuracy of Test: 0.5993
Precision of Test: 0.6481
top_k_result: 0.7202

men
Accuracy of Test: 0.6780
Precision of Test: 0.7365
top_k_result: 0.7600
'''

In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
print(first_pred_disease_name, round(first[1]*100, 2),'%','\n',
      '동의어:', first_info['동의어'].values[0]+'\n',
      '진료과: ', first_info['진료과'].values[0]+'\n',
      '질병 설명:', first_info['정의'].values[0]
      )

소화성 궤양 98.82 % 
 동의어: 소화성 궤양,십이지장 궤양
 진료과:  소화기내과, 노년내과
 질병 설명: 위궤양은 위점막이 헐어서 궤양이 점막뿐만 아니라 근육층까지 침범한 것을 말합니다.


In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
first_info['진료과'].values[0]

'소화기내과, 노년내과'

In [ ]:
second_info = aa[aa['원 질병이름'] == second_pred_disease_name]
print(second_pred_disease_name, round(second[1]*100, 2),'%','\n',
      '동의어:', second_info['동의어'].values[0]+'\n',
      '진료과: ', second_info['진료과'].values[0]+'\n',
      '질병 설명:', second_info['정의'].values[0]
      )

공황장애 0.63 % 
 동의어: 공황발작, 우발적 발작성불안, 임소공포
 진료과:  정신건강의학과
 질병 설명: 공황 장애란 갑자기 극도의 두려움과 불안을 느끼는 불안 장애의 일종입니다.


In [ ]:
third_info = aa[aa['원 질병이름'] == third_pred_disease_name]
print(third_pred_disease_name, round(third[1]*100, 2),'%','\n',
      '동의어:', third_info['동의어'].values[0]+'\n',
      '진료과: ', third_info['진료과'].values[0]+'\n',
      '질병 설명:', third_info['정의'].values[0]
      )

약물에 의한 변비 0.52 % 
 동의어: 변비
 진료과:  소화기내과
 질병 설명: 변비는 대장의 연동 운동이 저하되어 원활한 배변 운동을 하지 못하는 질환을 의미합니다.


# **ver6:**

embedding_dim = 1024, dense 2400 -- 800, dropout 0.5
stopwords 똑같이 만들어주기

In [ ]:
'''
women


men

'''

In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
print(first_pred_disease_name, round(first[1]*100, 2),'%','\n',
      '동의어:', first_info['동의어'].values[0]+'\n',
      '진료과: ', first_info['진료과'].values[0]+'\n',
      '질병 설명:', first_info['정의'].values[0]
      )

소화성 궤양 95.91 % 
 동의어: 소화성 궤양,십이지장 궤양
 진료과:  소화기내과, 노년내과
 질병 설명: 위궤양은 위점막이 헐어서 궤양이 점막뿐만 아니라 근육층까지 침범한 것을 말합니다.


In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
first_info['진료과'].values[0]

'소화기내과, 노년내과'

In [ ]:
second_info = aa[aa['원 질병이름'] == second_pred_disease_name]
print(second_pred_disease_name, round(second[1]*100, 2),'%','\n',
      '동의어:', second_info['동의어'].values[0]+'\n',
      '진료과: ', second_info['진료과'].values[0]+'\n',
      '질병 설명:', second_info['정의'].values[0]
      )

양성돌발체위현훈 3.92 % 
 동의어: 양성 돌발성 체위 현훈증,양성 발작성 현기증,양성 체위성 현훈,양성자세현훈
 진료과:  이비인후과 
 질병 설명: 이석증은 주변이 빙글빙글 도는 듯한 심한 어지럼이 수초에서 1분 정도 지속되다가 저절로 좋아지는 일이 반복되는 증상으로, 원래 명칭은 양성 발작성 체위성 현훈입니다. 이석증은 어지럼의 가장 흔한 원인입니다.


In [ ]:
third_info = aa[aa['원 질병이름'] == third_pred_disease_name]
print(third_pred_disease_name, round(third[1]*100, 2),'%','\n',
      '동의어:', third_info['동의어'].values[0]+'\n',
      '진료과: ', third_info['진료과'].values[0]+'\n',
      '질병 설명:', third_info['정의'].values[0]
      )

위암 0.1 % 
 동의어: Gastric cancer,Malignant neoplasm of stomach,cancer of the stomach,위 악성종양,위선암,위선암종,위암종
 진료과:  소화기내과, 종양내과, 방사선종양학과, 위장관외과, 암병원
 질병 설명: 위에 발생하는 악성 종양에는 위 선암, 림프종, 위 점막하 종양, 평활 근육종 등이 있습니다. 이 중에서 위 선암이 98%를 차지합니다. 따라서 위암은 일반적으로 위 선암을 말합니다. 위암은 위의 점막에서 발생합니다. 시간이 지남에 따라 점막하층, 근육층, 장막하층, 장막층으로 침윤합니다. 위암은 점막 또는 점막하층을 따라 위 내에 넓게 퍼지기도 하고, 점막층에서 장막층을 향해 깊이 퍼지기도 합니다. 위 주변의 임파선을 따라서, 혹은 혈류의 파급에 의해 간, 폐, 뼈 등의 여러 부위로 퍼질 수 있습니다.


# **ver5:**

embedding_dim = 1024, dense 2400 -- 800, dropout 0.5
stopwords 똑같이 만들어주기

In [ ]:
'''
women
Accuracy of Test: 0.6480
Precision of Test: 0.7114
top_k_result: 0.7635

men
Accuracy of Test: 0.6820
Precision of Test: 0.7326
top_k_result: 0.7580

'''

In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
print(first_pred_disease_name, round(first[1]*100, 2),'%','\n',
      '동의어:', first_info['동의어'].values[0]+'\n',
      '진료과: ', first_info['진료과'].values[0]+'\n',
      '질병 설명:', first_info['정의'].values[0]
      )

요도염 49.4 % 
 동의어: 요도염
 진료과:  비뇨의학과
 질병 설명: 요도염은 성교에 의해 전파되며, 요도 분비물, 배뇨 시 통증, 요도의 소양감(간지러움) 등의 증상을 나타내는 요도의 염증성 질환입니다.


In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
first_info['진료과'].values[0]

'비뇨의학과'

In [ ]:
second_info = aa[aa['원 질병이름'] == second_pred_disease_name]
print(second_pred_disease_name, round(second[1]*100, 2),'%','\n',
      '동의어:', second_info['동의어'].values[0]+'\n',
      '진료과: ', second_info['진료과'].values[0]+'\n',
      '질병 설명:', second_info['정의'].values[0]
      )

식도암 37.49 % 
 동의어: 식도암종
 진료과:  소화기내과, 종양내과, 영상의학과, 방사선종양학과, 신경외과, 이비인후과, 흉부외과, 위장관외과
 질병 설명: 식도암은 식도에 생긴 암입니다. 식도암은 위치에 따라 경부 식도암, 흉부 식도암, 위-식도 연결부위암으로 구분되고, 세포의 형태에 따라 편평세포암, 선암, 육종, 림프종, 흑색종 등으로 구분됩니다. 우리나라에서 많이 발생하는 식도암은 편평세포암으로, 전체 식도암의 95% 정도를 차지합니다. 편평세포암은 식도 점막의 상피세포에서 생기는 암으로, 대개 식도의 중부와 하부에 발생합니다. 선암은 식도의 분비샘 조직에서 생기는 암으로, 대개 위 근처인 하부 식도에서 발생합니다.


In [ ]:
third_info = aa[aa['원 질병이름'] == third_pred_disease_name]
print(third_pred_disease_name, round(third[1]*100, 2),'%','\n',
      '동의어:', third_info['동의어'].values[0]+'\n',
      '진료과: ', third_info['진료과'].values[0]+'\n',
      '질병 설명:', third_info['정의'].values[0]
      )

역류성 식도염 6.39 % 
 동의어: 역류성 식도염,위 식도 역류질환
 진료과:  가정의학과, 소화기내과, 위장관외과, 소아소화기영양과
 질병 설명: 위식도 역류성 질환은 위 내용물이 소량씩 식도로 역류하고, 이러한 역류의 과정이 반복되어 식도 점막이 손상되어 염증이 나타나는 질환을 의미합니다. 이는 위와 식도 사이에서 위의 내용물이 식도로 올라오는 것을 막아 주는 조임쇠 역할을 하는 괄약근에 이상이 생겨 발생합니다.


# **ver4:**

embedding_dim = 1024, dense 2400 -- 800, dropout 0.5 (여자는 dropout 0.25)

stopwords 똑같이 만들어주기

In [ ]:
'''
women
Accuracy of Test: 0.6318
Precision of Test: 0.6943
top_k_result: 0.7401

men
Accuracy of Test: 0.7180
Precision of Test: 0.7601
top_k_result: 0.8160
--> 전체적으로 다 오른 것을 볼 수 있음
'''

In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
print(first_pred_disease_name, round(first[1]*100, 2),'%','\n',
      '동의어:', first_info['동의어'].values[0]+'\n',
      '진료과: ', first_info['진료과'].values[0]+'\n',
      '질병 설명:', first_info['정의'].values[0]
      )

소화성 궤양 97.19 % 
 동의어: 소화성 궤양,십이지장 궤양
 진료과:  소화기내과, 노년내과
 질병 설명: 위궤양은 위점막이 헐어서 궤양이 점막뿐만 아니라 근육층까지 침범한 것을 말합니다.


In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
first_info['진료과'].values[0]

'소화기내과, 노년내과'

In [ ]:
second_info = aa[aa['원 질병이름'] == second_pred_disease_name]
print(second_pred_disease_name, round(second[1]*100, 2),'%','\n',
      '동의어:', second_info['동의어'].values[0]+'\n',
      '진료과: ', second_info['진료과'].values[0]+'\n',
      '질병 설명:', second_info['정의'].values[0]
      )

감염성 대장염 2.17 % 
 동의어: 대장염
 진료과:  소화기내과
 질병 설명: 감염성 대장염은 바이러스나 세균 등에 의하여 대장에 발생하는 감염입니다.


In [ ]:
third_info = aa[aa['원 질병이름'] == third_pred_disease_name]
print(third_pred_disease_name, round(third[1]*100, 2),'%','\n',
      '동의어:', third_info['동의어'].values[0]+'\n',
      '진료과: ', third_info['진료과'].values[0]+'\n',
      '질병 설명:', third_info['정의'].values[0]
      )

불안 / 우울 0.35 % 
 동의어: 우울병,우울장애,울병,주요 우울장애
 진료과:  정신건강의학과
 질병 설명: 우울증은 생각의 내용, 사고 과정, 동기, 의욕, 관심, 행동, 수면, 신체 활동 등 전반적인 정신 기능이 지속적으로 저하되어 일상생활에도 악영향을 미치는 상태를 의미합니다.


# **ver3:**

embedding_dim = 1024, dense 256 -- 64, dropout 0.5
stopwords 똑같이 만들어주기

In [ ]:
'''
women
Accuracy of Test: 0.6390
Precision of Test: 0.7478
top_k_result: 0.7238

men
Accuracy of Test: 0.6860
Precision of Test: 0.8076
top_k_result: 0.7880
수치는 올랐으나 --> 질병 예측의 전체적인 정확도 내려감 
'''

In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
print(first_pred_disease_name, round(first[1]*100, 2),'%','\n',
      '동의어:', first_info['동의어'].values[0]+'\n',
      '진료과: ', first_info['진료과'].values[0]+'\n',
      '질병 설명:', first_info['정의'].values[0]
      )

소화성 궤양 57.36 % 
 동의어: 소화성 궤양,십이지장 궤양
 진료과:  소화기내과, 노년내과
 질병 설명: 위궤양은 위점막이 헐어서 궤양이 점막뿐만 아니라 근육층까지 침범한 것을 말합니다.


In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
first_info['진료과'].values[0]

'소화기내과, 노년내과'

In [ ]:
second_info = aa[aa['원 질병이름'] == second_pred_disease_name]
print(second_pred_disease_name, round(second[1]*100, 2),'%','\n',
      '동의어:', second_info['동의어'].values[0]+'\n',
      '진료과: ', second_info['진료과'].values[0]+'\n',
      '질병 설명:', second_info['정의'].values[0]
      )

기관지 확장증 8.34 % 
 동의어: 기관지확장병
 진료과:  호흡기내과
 질병 설명: 기관지확장증은 폐 속의 큰 기도인 기관지가 과거의 심한 호흡기계 염증으로 손상을 입어 영구적으로 확장됨으로써 객담 배출 기능이 약해진 상태를 의미합니다.


In [ ]:
third_info = aa[aa['원 질병이름'] == third_pred_disease_name]
print(third_pred_disease_name, round(third[1]*100, 2),'%','\n',
      '동의어:', third_info['동의어'].values[0]+'\n',
      '진료과: ', third_info['진료과'].values[0]+'\n',
      '질병 설명:', third_info['정의'].values[0]
      )

식도정맥류 7.1 % 
 동의어: Esophageal Varix
 진료과:  소화기내과
 질병 설명: 식도 정맥류는 식도에 있는 정맥이 혹처럼 부풀어 오르는 질환입니다. 이 질환은 간경변증의 주요 합병증입니다. 식도 정맥류는 출혈이 발생할 경우 사망률이 12~22%에 달하는 응급 질환입니다. 간경변증 환자의 주요 사망 원인 중 하나입니다.


# **ver2:**

embedding_dim = 1024, dense 256 -- 64
stopwords 똑같이 만들어주기

In [ ]:
'''
women
Accuracy of Test: 0.5650
Precision of Test: 0.7311
top_k_result: 0.6787

men
Accuracy of Test: 0.6580
Precision of Test: 0.7529
top_k_result: 0.7540
'''

In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
print(first_pred_disease_name, round(first[1]*100, 2),'%','\n',
      '동의어:', first_info['동의어'].values[0]+'\n',
      '진료과: ', first_info['진료과'].values[0]+'\n',
      '질병 설명:', first_info['정의'].values[0]
      )

역류성 식도염 60.22 % 
 동의어: 역류성 식도염,위 식도 역류질환
 진료과:  가정의학과, 소화기내과, 위장관외과, 소아소화기영양과
 질병 설명: 위식도 역류성 질환은 위 내용물이 소량씩 식도로 역류하고, 이러한 역류의 과정이 반복되어 식도 점막이 손상되어 염증이 나타나는 질환을 의미합니다. 이는 위와 식도 사이에서 위의 내용물이 식도로 올라오는 것을 막아 주는 조임쇠 역할을 하는 괄약근에 이상이 생겨 발생합니다.


In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
first_info['진료과'].values[0]

'가정의학과, 소화기내과, 위장관외과, 소아소화기영양과'

In [ ]:
second_info = aa[aa['원 질병이름'] == second_pred_disease_name]
print(second_pred_disease_name, round(second[1]*100, 2),'%','\n',
      '동의어:', second_info['동의어'].values[0]+'\n',
      '진료과: ', second_info['진료과'].values[0]+'\n',
      '질병 설명:', second_info['정의'].values[0]
      )

소화성 궤양 19.02 % 
 동의어: 소화성 궤양,십이지장 궤양
 진료과:  소화기내과, 노년내과
 질병 설명: 위궤양은 위점막이 헐어서 궤양이 점막뿐만 아니라 근육층까지 침범한 것을 말합니다.


In [ ]:
third_info = aa[aa['원 질병이름'] == third_pred_disease_name]
print(third_pred_disease_name, round(third[1]*100, 2),'%','\n',
      '동의어:', third_info['동의어'].values[0]+'\n',
      '진료과: ', third_info['진료과'].values[0]+'\n',
      '질병 설명:', third_info['정의'].values[0]
      )

식도암 6.05 % 
 동의어: 식도암종
 진료과:  소화기내과, 종양내과, 영상의학과, 방사선종양학과, 신경외과, 이비인후과, 흉부외과, 위장관외과
 질병 설명: 식도암은 식도에 생긴 암입니다. 식도암은 위치에 따라 경부 식도암, 흉부 식도암, 위-식도 연결부위암으로 구분되고, 세포의 형태에 따라 편평세포암, 선암, 육종, 림프종, 흑색종 등으로 구분됩니다. 우리나라에서 많이 발생하는 식도암은 편평세포암으로, 전체 식도암의 95% 정도를 차지합니다. 편평세포암은 식도 점막의 상피세포에서 생기는 암으로, 대개 식도의 중부와 하부에 발생합니다. 선암은 식도의 분비샘 조직에서 생기는 암으로, 대개 위 근처인 하부 식도에서 발생합니다.


# **ver1:**

embedding_dim = 1024, dense 256 -- 64

In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
print(first_pred_disease_name, round(first[1]*100, 2),'%','\n',
      '동의어:', first_info['동의어'].values[0]+'\n',
      '진료과: ', first_info['진료과'].values[0]+'\n',
      '질병 설명:', first_info['정의'].values[0]
      )

메켈 게실 34.43 % 
 동의어: 업데이트 예정
 진료과:  소화기내과
 질병 설명: 메켈 게실이란 태생 초기 태아의 혈액 보급로인 제장관막관이 출생 전에 퇴화되지 않아 회장 벽에서 60~100cm 정도로 돌출된 비정상적인 낭을 형성하는 질환입니다. 소화기 선천 기형 중 가장 흔한 질병 중 하나입니다.


In [ ]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
first_info['진료과'].values[0]

'소화기내과'

In [ ]:
second_info = aa[aa['원 질병이름'] == second_pred_disease_name]
print(second_pred_disease_name, round(second[1]*100, 2),'%','\n',
      '동의어:', second_info['동의어'].values[0]+'\n',
      '진료과: ', second_info['진료과'].values[0]+'\n',
      '질병 설명:', second_info['정의'].values[0]
      )

크롬세포친화종 18.33 % 
 동의어: 크롬친화세포종
 진료과:  내분비내과
 질병 설명: 갈색세포종은 주로 부신수질에서 발생하는 종양을 말합니다.


In [ ]:
third_info = aa[aa['원 질병이름'] == third_pred_disease_name]
print(third_pred_disease_name, round(third[1]*100, 2),'%','\n',
      '동의어:', third_info['동의어'].values[0]+'\n',
      '진료과: ', third_info['진료과'].values[0]+'\n',
      '질병 설명:', third_info['정의'].values[0]
      )

뇌척수액 비루 16.95 % 
 동의어: 업데이트 예정
 진료과:  신경외과
 질병 설명: 뇌척수액 비루란 뇌척수액이 마치 콧물처럼 코로 새어나오는 질환입니다. 맑고 물 같은 콧물이 계속 나오며, 아랫배에 힘을 주었을 때 더 많이 나온다면 뇌척수액비루일 가능성이 있습니다.
